#### Контроль Квот по областям (Содержание<a class="anchor" id="chapter0"></a>)


* [Импорт библиотек](#chapter1)
* [Форма отчета](#chapter2)
* [Выгружаем квоты](#chapter3)
    * [DataFrame с квотами по тк4](#selection_3_1) 
    * [Общие системные квоты](#selection_3_2)
    * [Проверка на изменение квот по сумме](#selection_3_3)
* [Наполнение по магазинам для листа "ТТ с переквотой + по уровням"](#chapter4)
* [Расчет доли ТТ с переквотой](#chapter5)
* [Расчет абсолютных переквот](#chapter7)
* [Расчет переквот по взвесу](#chapter8)
* [Расчет переквот по взвесу по обл](#chapter811)
* [Вывод на листы Excel](#chapter9)   
    * [Стили](#selection_9_1)
    * [Мол.гор.йогурты](#selection_9_2)
    * [Мол.гор.тради](#selection_9_3)
    * [Мясная гастрономия](#selection_9_4)

 * [The End](#chapter10)

In [1]:
# ВЫВОД В ФАЙЛ EXCEL:

path_mgi_data=r"C:\Users\VYuYaroshenko\Desktop\Пайтон Молочка\КонтрольКвот_по_областям_МГЙ.xlsx"
path_mgt_data= r"C:\Users\VYuYaroshenko\Desktop\Пайтон Молочка\КонтрольКвот_по_областям_МГТ.xlsx"

In [2]:
# текущая дата
#import datetime
#current_time = datetime.datetime.now()
#print('Дата формирования отчета: ', current_time, '.', current_time.year)

# текущая дата
from datetime import date
current_date = date.today()
current_date=str(current_date)
curr_date = 'Дата формирования отчета: ' + current_date
curr_date

'Дата формирования отчета: 2025-03-19'

<div class="alert alert-success">
    <b>✅ Импорт библиотек
        </div> <a class="anchor" id="chapter1"></a>

[вначало](#chapter0)

In [3]:
import pyodbc #подключение к SQL Server
import pathlib #Работа с файлами и путями к ним
import pandas as pd #Работа с табличными данными
import datetime #Работа с датами
import warnings  #Выключение предупреждений
from datetime import datetime

import openpyxl #Работа с файлами excel
from openpyxl.worksheet import worksheet
import os
from pandas.io.excel import ExcelWriter
from openpyxl import Workbook
from openpyxl.styles import (
    PatternFill, 
    NamedStyle, 
    Font, 
    Border, 
    Side, 
    Alignment, 
    GradientFill)

from openpyxl.worksheet.filters import (
    FilterColumn,
    CustomFilter,
    CustomFilters,
    DateGroupItem,
    #BlankFilter,
    Filters,
    )
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule

#import xlswriter

In [4]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') #привязали библиотеку, которая помагет игнорировать несущественные ошибки

Driver='ODBC Driver 17 for SQL Server'
Server=r'srv-dwh-lsn,15025'
Database=['dwh','Reps','presentation']
# 'dwh' = Database[0]
# 'Reps' = Database[1]
# 'presentation' = Database[2]

def get_adw_connection():
    # Установка соединения 
    connection = pyodbc.connect('Driver=' + Driver + ';\
                                 Server=' + Server + ';\
                                 Database=' + Database[0] + ';\
                                 Trusted_Connection=yes')
    return connection

<div class="alert alert-success">
    <b>✅ Форма отчета
        </div> <a class="anchor" id="chapter2"></a>

[вначало](#chapter0)

In [5]:
%%time
query = """

with 
shops as
    (SELECT [SiteId] ,[SiteExtId] 
		,case when [level6_descr] in ('M','S','XL','L') then 'GO' --when [level6_descr]='DARKSTORE' then 'DARKSTORE' 
        when [level6_descr]='TOWER' then 'TOWER' else 'DIXY' end  'format' , [level4_descr] 'obl', [level3_descr] 'fo'  
    FROM [DWH].[dbo].[DNodes]  --i on i.SiteID = n.SiteID
    WHERE 1=1 
        --and not level4_descr in ('СВЕРДЛОВСКАЯ ОБЛАСТЬ','ТЮМЕНСКАЯ ОБЛАСТЬ','ЧЕЛЯБИНСКАЯ ОБЛАСТЬ','АРХАНГЕЛЬСКАЯ ОБЛАСТЬ','ВОЛОГОДСКАЯ ОБЛАСТЬ','КАРЕЛИЯ','МУРМАНСКАЯ ОБЛАСТЬ','ПСКОВСКАЯ ОБЛАСТЬ') 
        and LegalEntityName like 'АО "ДИКСИ ЮГ"' --and [ClosedDate] is NULL and level2_descr='ДИКСИ' 
	)
,am as
    (SELECT n.SiteId, n.SiteExtId, 
        case n.fo when 'ЦЕНТРАЛЬНЫЙ' Then 'ЦФО' when 'СЕВЕРО-ЗАПАДНЫЙ' Then 'СЗФО' when 'УРАЛЬСКИЙ' Then 'УФО' end 'FO', 
        n.format 'Format', n.obl 'Devision', atr.PlngDescription 'Plng', atr.PlngSize 'PLNG_SIZE', a.ItemId
    FROM dwh.gld.TypeSettingMatrixPeriod a with(nolock) 
        join dwh.gld.Attributes atr with(nolock) on atr.AttributeId = a.AttributeId 
        JOIN shops n with(nolock) on n.[SiteId] = a.SiteId 
    where 1=1 
		and cast(convert(varchar,getdate(),112)as int) between DateFrom and Dateto  
        and atr.[PlngExtId] in (122, 149,111, 148) --МОЛ.ГОР ЙОГУРТЫ          --122, --МОЛ.ГОР ТРАДИЦ 148, --МОЛ.ГОР ДЕТПИТ        --111 --МЯСНАЯ ГАСТРОНОМИЯ
     )		 
,plng as
    (SELECT a.SiteId, a.SiteExtId, a.FO, a.Format, a.Devision, a.ItemId, a.Plng, a.PLNG_SIZE, 
        LEFT(a.PLNG_SIZE,1) 'Claster',SUBSTRING(a.PLNG_SIZE,2,6) 'Size' ,p.Level4_desc 'levels'
    FROM AM a with(nolock) JOIN dwh.dbo.DProducts p with(nolock) on a.ItemId = p.ItemId 
	)
	   	--Планограмма	Уровень	ФО	ОБЛАСТЬ	КЛАСТЕР	Размер	Формат

SELECT Plng 'Планограмма'
	,case when levels in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else levels end 'Уровень'
	,FO 'ФО', Devision 'ОБЛАСТЬ',Claster 'КЛАСТЕР',Size 'Размер',Format 'Формат' 
FROM plng --where Format ='T'
group by    Plng ,levels ,FO , Devision ,Claster,Size,Format 

"""
gp=get_adw_connection()
forme = pd.read_sql(query, con=gp)
gp.close()

CPU times: total: 250 ms
Wall time: 31.7 s


In [6]:
forme['Формат'].unique()

array(['GO', 'DIXY', 'TOWER'], dtype=object)

In [7]:
formes=forme
formes.sample(4)
formes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2867 entries, 0 to 2866
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  2867 non-null   object
 1   Уровень      2867 non-null   object
 2   ФО           2867 non-null   object
 3   ОБЛАСТЬ      2867 non-null   object
 4   КЛАСТЕР      2867 non-null   object
 5   Размер       2867 non-null   object
 6   Формат       2867 non-null   object
dtypes: object(7)
memory usage: 156.9+ KB


In [8]:
#formes_mgt = pd.read_excel(pth, sheet_name='МГТ')
#formes_mgt['Размер']=formes_mgt['Размер'].astype(str)
#formes_mgt = formes_mgt.rename(columns = {'Plng':'Планограмма', 'Format':'Формат'})
#formes_mgt.info() 

**МОЛ.ГОР ЙОГУРТЫ** <a class="anchor" id="selection_2_2"></a>

In [9]:
formes_mgi=formes.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"')
formes_mgi.sample(5) 

,Планограмма,Уровень,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Формат
564,МОЛ.ГОР ЙОГУРТЫ,РАСТИТЕЛЬНЫЕ ПРОДУКТЫ,ЦФО,ТУЛЬСКАЯ ОБЛАСТЬ,M,1,DIXY
547,МОЛ.ГОР ЙОГУРТЫ,РАСТИТЕЛЬНЫЕ ПРОДУКТЫ,ЦФО,СМОЛЕНСКАЯ ОБЛАСТЬ,M,1,DIXY
326,МОЛ.ГОР ЙОГУРТЫ,ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,ЦФО,МОСКВА,L,2,DIXY
742,МОЛ.ГОР ЙОГУРТЫ,СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ,ЦФО,ТУЛЬСКАЯ ОБЛАСТЬ,T,4,DIXY
83,МОЛ.ГОР ЙОГУРТЫ,ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,СЗФО,НОВГОРОДСКАЯ ОБЛАСТЬ,M,1,DIXY


In [10]:
formes_mgi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 692 entries, 62 to 753
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  692 non-null    object
 1   Уровень      692 non-null    object
 2   ФО           692 non-null    object
 3   ОБЛАСТЬ      692 non-null    object
 4   КЛАСТЕР      692 non-null    object
 5   Размер       692 non-null    object
 6   Формат       692 non-null    object
dtypes: object(7)
memory usage: 43.2+ KB


**МОЛ.ГОР ТРАДИЦ** 

In [11]:
formes_mgt=formes.query('Планограмма=="МОЛ.ГОР ТРАДИЦ"')
formes_mgt.sample(5) 

,Планограмма,Уровень,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Формат
1335,МОЛ.ГОР ТРАДИЦ,СМЕТАНА,ЦФО,РЯЗАНСКАЯ ОБЛАСТЬ,M,4,DIXY
900,МОЛ.ГОР ТРАДИЦ,КИСЛОМОЛОЧНЫЕ ПРОДУКТЫ,ЦФО,ТВЕРСКАЯ ОБЛАСТЬ,M,4,DIXY
1264,МОЛ.ГОР ТРАДИЦ,СМЕТАНА,ЦФО,ИВАНОВСКАЯ ОБЛАСТЬ,H,1,DIXY
1175,МОЛ.ГОР ТРАДИЦ,"МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.",ЦФО,СМОЛЕНСКАЯ ОБЛАСТЬ,M,2,DIXY
937,МОЛ.ГОР ТРАДИЦ,"МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.",СЗФО,ПСКОВСКАЯ ОБЛАСТЬ,L,4,DIXY


In [12]:
formes_mgt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 791 entries, 754 to 1544
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  791 non-null    object
 1   Уровень      791 non-null    object
 2   ФО           791 non-null    object
 3   ОБЛАСТЬ      791 non-null    object
 4   КЛАСТЕР      791 non-null    object
 5   Размер       791 non-null    object
 6   Формат       791 non-null    object
dtypes: object(7)
memory usage: 49.4+ KB


**МЯСНАЯ ГАСТРОНОМИЯ** 

In [13]:
formes_mg=formes.query('Планограмма=="МЯСНАЯ ГАСТРОНОМИЯ"')
formes_mg.sample(5) 

,Планограмма,Уровень,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Формат
1563,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,L,5,DIXY
2441,МЯСНАЯ ГАСТРОНОМИЯ,"СОСИСКИ, САРДЕЛЬКИ",ЦФО,ОРЛОВСКАЯ ОБЛАСТЬ,L,8,DIXY
2172,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,ЦФО,ТУЛЬСКАЯ ОБЛАСТЬ,L,6,DIXY
1775,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,T,8,DIXY
2292,МЯСНАЯ ГАСТРОНОМИЯ,"СОСИСКИ, САРДЕЛЬКИ",ЦФО,БРЯНСКАЯ ОБЛАСТЬ,L,5,DIXY


In [14]:
formes_mg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1322 entries, 1545 to 2866
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  1322 non-null   object
 1   Уровень      1322 non-null   object
 2   ФО           1322 non-null   object
 3   ОБЛАСТЬ      1322 non-null   object
 4   КЛАСТЕР      1322 non-null   object
 5   Размер       1322 non-null   object
 6   Формат       1322 non-null   object
dtypes: object(7)
memory usage: 82.6+ KB


<div class="alert alert-success"> <b>✅ Выгружаем квоты </b></div> <a class="anchor" id="chapter3"></a>

[вначало](#chapter0)

**DataFrame с квотами по тк4**  <a class="anchor" id="selection_3_1"></a>

**МЯСНАЯ ГАСТРОНОМИЯ** 

In [15]:
quots_mgs=  [['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 1, 18, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 1, 33, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 1, 22, 'DIXY'],
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 2, 23, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 2, 45, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 2, 28, 'DIXY'],
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 3, 24, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 3, 47, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 3, 30, 'DIXY'],
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 4, 29, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 4, 57, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 4, 36, 'DIXY'],
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 5, 38, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 5, 74, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 5, 46, 'DIXY'],   
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 6, 43, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 6, 84, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 6, 52, 'DIXY'],   

             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 7, 55, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 7, 99, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 7, 56, 'DIXY'],   
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 8, 64, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 8, 108, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 8, 58, 'DIXY'],  
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 9, 66, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 9, 113, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 9, 61, 'DIXY'],               
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 10, 75, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 10, 122, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 10, 61, 'DIXY'],   
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 11, 90, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 11, 139, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 11, 66, 'DIXY'],                

             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 12, 94, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 12, 148, 'DIXY'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 12, 71, 'DIXY'],   
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 20, 15, 'GO'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 20, 26, 'GO'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 20, 15, 'GO'],
             
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'ДЕЛИКАТЕСЫ', 10, 13, 'TOWER'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'КОЛБАСЫ', 10, 24, 'TOWER'],
             ['МЯСНАЯ ГАСТРОНОМИЯ', 'СОСИСКИ, САРДЕЛЬКИ', 10, 13, 'TOWER']]           

quots_mgs=pd.DataFrame(quots_mgs, columns=['Plng', 'Уровень', 'Размер',  'Count_Quots', 'Format'])     

In [16]:
quots_mgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Plng         42 non-null     object
 1   Уровень      42 non-null     object
 2   Размер       42 non-null     int64 
 3   Count_Quots  42 non-null     int64 
 4   Format       42 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.8+ KB


In [17]:
quots_mg=quots_mgs
quots_mg.sample(5)

,Plng,Уровень,Размер,Count_Quots,Format
27,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,10,75,DIXY
6,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,3,24,DIXY
14,МЯСНАЯ ГАСТРОНОМИЯ,"СОСИСКИ, САРДЕЛЬКИ",5,46,DIXY
11,МЯСНАЯ ГАСТРОНОМИЯ,"СОСИСКИ, САРДЕЛЬКИ",4,36,DIXY
19,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,7,99,DIXY


In [18]:
quots_mg.query('Format=="GO"')

,Plng,Уровень,Размер,Count_Quots,Format
36,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,20,15,GO
37,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,20,26,GO
38,МЯСНАЯ ГАСТРОНОМИЯ,"СОСИСКИ, САРДЕЛЬКИ",20,15,GO


In [19]:
quots_mg['Count_Quots']=quots_mg['Count_Quots'].fillna(0)
quots_mg['Count_Quots']=quots_mg['Count_Quots'].astype(int)
quots_mg['Размер']=quots_mg['Размер'].astype(str)
quots_mg = quots_mg.rename(columns = {'Plng':'Планограмма', 'Format':'Формат'})
#quots_mg#.sample(5) # 64rows

formes_mg_quots = pd.merge(formes_mg, quots_mg, on = ['Планограмма', 'Уровень', 'Размер', 'Формат'], how = 'left')
formes_mg_quots['Count_Quots']=formes_mg_quots['Count_Quots'].fillna(0)
formes_mg_quots['Count_Quots']=formes_mg_quots['Count_Quots'].astype(int)
formes_mg_quots = formes_mg_quots.rename(columns = {'Count_Quots':'Квота'})
formes_mg_quots.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1322 entries, 0 to 1321
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  1322 non-null   object
 1   Уровень      1322 non-null   object
 2   ФО           1322 non-null   object
 3   ОБЛАСТЬ      1322 non-null   object
 4   КЛАСТЕР      1322 non-null   object
 5   Размер       1322 non-null   object
 6   Формат       1322 non-null   object
 7   Квота        1322 non-null   int32 
dtypes: int32(1), object(7)
memory usage: 87.8+ KB


In [20]:
formes_mg_quots.sample(5)

,Планограмма,Уровень,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Формат,Квота
204,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,L,2,DIXY,23
1084,МЯСНАЯ ГАСТРОНОМИЯ,ЯЙЦА,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M,9,DIXY,0
80,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,СЗФО,САНКТ-ПЕТЕРБУРГ,T,5,DIXY,38
585,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,ЦФО,СМОЛЕНСКАЯ ОБЛАСТЬ,H,10,DIXY,122
1104,МЯСНАЯ ГАСТРОНОМИЯ,ЯЙЦА,ЦФО,ВЛАДИМИРСКАЯ ОБЛАСТЬ,M,9,DIXY,0


In [21]:
formes_mg_quots.query('Формат=="GO"')

,Планограмма,Уровень,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Формат,Квота
0,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,G,20,GO,15
48,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,СЗФО,САНКТ-ПЕТЕРБУРГ,G,20,GO,15
151,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,ЦФО,МОСКВА,G,20,GO,15
188,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,G,20,GO,15
189,МЯСНАЯ ГАСТРОНОМИЯ,ДЕЛИКАТЕСЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,G,21,GO,0
332,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,G,20,GO,26
380,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,СЗФО,САНКТ-ПЕТЕРБУРГ,G,20,GO,26
483,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,ЦФО,МОСКВА,G,20,GO,26
520,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,G,20,GO,26
521,МЯСНАЯ ГАСТРОНОМИЯ,КОЛБАСЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,G,21,GO,0


**МОЛ.ГОР ТРАДИЦ** <a class="anchor" id="selection_3_1_1"></a>

In [22]:
quots_mgts= [['МОЛ.ГОР ТРАДИЦ', 'ТВОРОЖНЫЕ ИЗДЕЛИЯ', 1, 33, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'СМЕТАНА', 1, 16, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.', 1, 18, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'КИСЛОМОЛОЧНЫЕ ПРОДУКТЫ', 1,  28, 'DIXY'],
             
             ['МОЛ.ГОР ТРАДИЦ', 'ТВОРОЖНЫЕ ИЗДЕЛИЯ', 2, 41, 'DIXY'],             
             ['МОЛ.ГОР ТРАДИЦ', 'СМЕТАНА', 2, 19, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.', 2, 19, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'КИСЛОМОЛОЧНЫЕ ПРОДУКТЫ', 2,  36, 'DIXY'],
             
             ['МОЛ.ГОР ТРАДИЦ', 'ТВОРОЖНЫЕ ИЗДЕЛИЯ', 3, 50, 'DIXY'],             
             ['МОЛ.ГОР ТРАДИЦ', 'СМЕТАНА', 3, 23, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.', 3, 22, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'КИСЛОМОЛОЧНЫЕ ПРОДУКТЫ', 3,  40, 'DIXY'],           
             
             ['МОЛ.ГОР ТРАДИЦ', 'ТВОРОЖНЫЕ ИЗДЕЛИЯ', 4, 64, 'DIXY'],             
             ['МОЛ.ГОР ТРАДИЦ', 'СМЕТАНА', 4, 26, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.', 4, 26, 'DIXY'],
             ['МОЛ.ГОР ТРАДИЦ', 'КИСЛОМОЛОЧНЫЕ ПРОДУКТЫ', 4,  46, 'DIXY'],  
             
             ['МОЛ.ГОР ТРАДИЦ', 'ТВОРОЖНЫЕ ИЗДЕЛИЯ', 20, 13, 'GO'],             
             ['МОЛ.ГОР ТРАДИЦ', 'СМЕТАНА', 20, 7, 'GO'],
             ['МОЛ.ГОР ТРАДИЦ', 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.', 20, 7, 'GO'],
             ['МОЛ.ГОР ТРАДИЦ', 'КИСЛОМОЛОЧНЫЕ ПРОДУКТЫ', 20,  11, 'GO'],
             
             ['МОЛ.ГОР ТРАДИЦ', 'ТВОРОЖНЫЕ ИЗДЕЛИЯ', 50, 17, 'TOWER'], #15            
             ['МОЛ.ГОР ТРАДИЦ', 'СМЕТАНА', 50, 4, 'TOWER'], #6
             ['МОЛ.ГОР ТРАДИЦ', 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.', 50, 3, 'TOWER'], #7
             ['МОЛ.ГОР ТРАДИЦ', 'КИСЛОМОЛОЧНЫЕ ПРОДУКТЫ', 50,  16, 'TOWER']]   #12        

quots_mgts=pd.DataFrame(quots_mgts, columns=['Plng', 'Уровень', 'Размер',  'Count_Quots', 'Format'])                     

In [23]:
quots_mgts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Plng         24 non-null     object
 1   Уровень      24 non-null     object
 2   Размер       24 non-null     int64 
 3   Count_Quots  24 non-null     int64 
 4   Format       24 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.1+ KB


In [24]:
quots_mgt=quots_mgts
quots_mgt.sample(5)

,Plng,Уровень,Размер,Count_Quots,Format
10,МОЛ.ГОР ТРАДИЦ,"МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.",3,22,DIXY
9,МОЛ.ГОР ТРАДИЦ,СМЕТАНА,3,23,DIXY
0,МОЛ.ГОР ТРАДИЦ,ТВОРОЖНЫЕ ИЗДЕЛИЯ,1,33,DIXY
16,МОЛ.ГОР ТРАДИЦ,ТВОРОЖНЫЕ ИЗДЕЛИЯ,20,13,GO
13,МОЛ.ГОР ТРАДИЦ,СМЕТАНА,4,26,DIXY


In [25]:
formes_mgt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 791 entries, 754 to 1544
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  791 non-null    object
 1   Уровень      791 non-null    object
 2   ФО           791 non-null    object
 3   ОБЛАСТЬ      791 non-null    object
 4   КЛАСТЕР      791 non-null    object
 5   Размер       791 non-null    object
 6   Формат       791 non-null    object
dtypes: object(7)
memory usage: 49.4+ KB


In [26]:
formes_mgt['Формат'].unique()

array(['GO', 'DIXY', 'TOWER'], dtype=object)

In [27]:
#quots_mgt = pd.read_excel(pthh, sheet_name='МГЙ')
quots_mgt['Count_Quots']=quots_mgt['Count_Quots'].fillna(0)
quots_mgt['Count_Quots']=quots_mgt['Count_Quots'].astype(int)
quots_mgt['Размер']=quots_mgt['Размер'].astype(str)
quots_mgt = quots_mgt.rename(columns = {'Plng':'Планограмма', 'Format':'Формат'})
#quots_mgt#.sample(5) # 64rows

In [28]:
quots_mgt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  24 non-null     object
 1   Уровень      24 non-null     object
 2   Размер       24 non-null     object
 3   Count_Quots  24 non-null     int32 
 4   Формат       24 non-null     object
dtypes: int32(1), object(4)
memory usage: 992.0+ bytes


In [29]:
quots_mgt['Формат'].unique()

array(['DIXY', 'GO', 'TOWER'], dtype=object)

In [30]:
formes_mgt_quots = pd.merge(formes_mgt, quots_mgt, on = ['Планограмма', 'Уровень', 'Размер', 'Формат'], how = 'left')
formes_mgt_quots['Count_Quots']=formes_mgt_quots['Count_Quots'].fillna(0)
formes_mgt_quots['Count_Quots']=formes_mgt_quots['Count_Quots'].astype(int)
formes_mgt_quots = formes_mgt_quots.rename(columns = {'Count_Quots':'Квота'})
formes_mgt_quots.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 791 entries, 0 to 790
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  791 non-null    object
 1   Уровень      791 non-null    object
 2   ФО           791 non-null    object
 3   ОБЛАСТЬ      791 non-null    object
 4   КЛАСТЕР      791 non-null    object
 5   Размер       791 non-null    object
 6   Формат       791 non-null    object
 7   Квота        791 non-null    int32 
dtypes: int32(1), object(7)
memory usage: 52.5+ KB


In [31]:
# пропуски по квотам, нет квот в Дарксторе
#formes_missing = formes_mgt_quots[formes_mgt_quots['Count_Quots'].isna()]
#display(formes_missing)

**МОЛ.ГОР ЙОГУРТЫ** <a class="anchor" id="selection_3_1_2"></a>

In [32]:
quots_mgis= [['МОЛ.ГОР ЙОГУРТЫ', 'ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 1, 52, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ', 1, 42, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 1, 71, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'РАСТИТЕЛЬНЫЕ ПРОДУКТЫ', 1,  1, 'DIXY'],
             
             ['МОЛ.ГОР ЙОГУРТЫ', 'ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 2, 58, 'DIXY'],             
             ['МОЛ.ГОР ЙОГУРТЫ', 'СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ', 2, 45, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 2, 86, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'РАСТИТЕЛЬНЫЕ ПРОДУКТЫ', 2,  2, 'DIXY'],
             
             ['МОЛ.ГОР ЙОГУРТЫ', 'ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 3, 75, 'DIXY'],             
             ['МОЛ.ГОР ЙОГУРТЫ', 'СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ', 3, 57, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 3, 108, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'РАСТИТЕЛЬНЫЕ ПРОДУКТЫ', 3,  2, 'DIXY'],           
             
             ['МОЛ.ГОР ЙОГУРТЫ', 'ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 4, 82, 'DIXY'],             
             ['МОЛ.ГОР ЙОГУРТЫ', 'СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ', 4, 61, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 4, 122, 'DIXY'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'РАСТИТЕЛЬНЫЕ ПРОДУКТЫ', 4,  2, 'DIXY'],  
             
             ['МОЛ.ГОР ЙОГУРТЫ', 'ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 20, 16, 'GO'],             
             ['МОЛ.ГОР ЙОГУРТЫ', 'СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ', 20, 13, 'GO'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 20, 23, 'GO'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'РАСТИТЕЛЬНЫЕ ПРОДУКТЫ', 20,  1, 'GO'],
             
             ['МОЛ.ГОР ЙОГУРТЫ', 'ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 50, 34, 'TOWER'],             
             ['МОЛ.ГОР ЙОГУРТЫ', 'СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ', 50, 26, 'TOWER'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ', 50, 49, 'TOWER'],
             ['МОЛ.ГОР ЙОГУРТЫ', 'РАСТИТЕЛЬНЫЕ ПРОДУКТЫ', 50,  1, 'TOWER']]           

quots_mgis=pd.DataFrame(quots_mgis, columns=['Plng', 'Уровень', 'Размер',  'Count_Quots', 'Format']) 

In [33]:
quots_mgis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Plng         24 non-null     object
 1   Уровень      24 non-null     object
 2   Размер       24 non-null     int64 
 3   Count_Quots  24 non-null     int64 
 4   Format       24 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.1+ KB


In [34]:
quots_mgi=quots_mgis
quots_mgi.sample(5)

,Plng,Уровень,Размер,Count_Quots,Format
20,МОЛ.ГОР ЙОГУРТЫ,ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,50,34,TOWER
11,МОЛ.ГОР ЙОГУРТЫ,РАСТИТЕЛЬНЫЕ ПРОДУКТЫ,3,2,DIXY
15,МОЛ.ГОР ЙОГУРТЫ,РАСТИТЕЛЬНЫЕ ПРОДУКТЫ,4,2,DIXY
21,МОЛ.ГОР ЙОГУРТЫ,СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ,50,26,TOWER
10,МОЛ.ГОР ЙОГУРТЫ,ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,3,108,DIXY


In [35]:
#quots_mgi = pd.read_excel(pthh, sheet_name='МГЙ')
quots_mgi['Count_Quots']=quots_mgi['Count_Quots'].fillna(0)
quots_mgi['Count_Quots']=quots_mgi['Count_Quots'].astype(int)
quots_mgi['Размер']=quots_mgi['Размер'].astype(str)
quots_mgi = quots_mgi.rename(columns = {'Plng':'Планограмма', 'Format':'Формат'})
#quots_mgi#.sample(5) # 64rows

formes_mgi_quots = pd.merge(formes_mgi, quots_mgi, on = ['Планограмма', 'Уровень', 'Размер', 'Формат'], how = 'left')
formes_mgi_quots['Count_Quots']=formes_mgi_quots['Count_Quots'].fillna(0)
formes_mgi_quots['Count_Quots']=formes_mgi_quots['Count_Quots'].astype(int)
formes_mgi_quots = formes_mgi_quots.rename(columns = {'Count_Quots':'Квота'})
formes_mgi_quots.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 692 entries, 0 to 691
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  692 non-null    object
 1   Уровень      692 non-null    object
 2   ФО           692 non-null    object
 3   ОБЛАСТЬ      692 non-null    object
 4   КЛАСТЕР      692 non-null    object
 5   Размер       692 non-null    object
 6   Формат       692 non-null    object
 7   Квота        692 non-null    int32 
dtypes: int32(1), object(7)
memory usage: 46.0+ KB


In [36]:
formes_mgi_quots.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 692 entries, 0 to 691
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  692 non-null    object
 1   Уровень      692 non-null    object
 2   ФО           692 non-null    object
 3   ОБЛАСТЬ      692 non-null    object
 4   КЛАСТЕР      692 non-null    object
 5   Размер       692 non-null    object
 6   Формат       692 non-null    object
 7   Квота        692 non-null    int32 
dtypes: int32(1), object(7)
memory usage: 46.0+ KB


In [37]:
formes_mgi_quots.query('Квота==0')

,Планограмма,Уровень,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Формат,Квота
83,МОЛ.ГОР ЙОГУРТЫ,ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,ЦФО,МОСКВА,D,77,DIXY,0
101,МОЛ.ГОР ЙОГУРТЫ,ЛОЖКОВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,G,21,GO,0
256,МОЛ.ГОР ЙОГУРТЫ,ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,ЦФО,МОСКВА,D,77,DIXY,0
274,МОЛ.ГОР ЙОГУРТЫ,ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,G,21,GO,0
429,МОЛ.ГОР ЙОГУРТЫ,РАСТИТЕЛЬНЫЕ ПРОДУКТЫ,ЦФО,МОСКВА,D,77,DIXY,0
447,МОЛ.ГОР ЙОГУРТЫ,РАСТИТЕЛЬНЫЕ ПРОДУКТЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,G,21,GO,0
602,МОЛ.ГОР ЙОГУРТЫ,СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ,ЦФО,МОСКВА,D,77,DIXY,0
620,МОЛ.ГОР ЙОГУРТЫ,СНЕКОВАЯ МОЛОЧНАЯ ПРОДУКЦИЯ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,G,21,GO,0


In [38]:
#formes_mgi_quots.query('Формат=="TOWER"')

**Общие системные квоты** <a class="anchor" id="selection_3_2"></a>

In [39]:
%%time
query = """
SET NOCOUNT ON; 

drop table if exists #dates		
;with d as
	(select *, d.Year*100+Month 'MonthId', d.YearForWeek*100+WeekOfYear 'WeekId', d.YearForWeek*10000+WeekOfYear*100+DayOfWeek 'DayOfWeekId' 
	from dwh.dbo.dates d)	
select * into #dates from d	where 1=1 and DateId in (convert(nvarchar, getdate(), 112) )

drop table if exists #qotes	
select p.Decription as 'Plng',p.[PlanogrammExtId] as 'NumPlng' ,cast(PlngSize as nvarchar(max)) as 'Plng_Size'	,QTA_RG	,SiteFormat 'Формат'
into #qotes	
from 
	(select PlngExtId, SiteFormat, PlngSize, AttributeClass, AttributeValue 
	from dwh.gld.ItemAttributes_Quota_data	
	join #dates d on d.DateId between StartDate and EndDate) a

pivot(sum(AttributeValue) 
	for AttributeClass in(
		[QTA_RG])) pvt
	left join dwh.staging.Planogramms p on p.PlanogrammExtId = pvt.PlngExtId	

select * from #qotes where Plng in ('МОЛ.ГОР ЙОГУРТЫ', 'МОЛ.ГОР ТРАДИЦ', 'МЯСНАЯ ГАСТРОНОМИЯ')
order by 2, 5 desc

"""
gp=get_adw_connection()
quotss = pd.read_sql(query, con=gp)
gp.close()

CPU times: total: 15.6 ms
Wall time: 45.3 ms


In [40]:
quots_all=quotss
quots_all.head()

,Plng,NumPlng,Plng_Size,QTA_RG,Формат
0,МЯСНАЯ ГАСТРОНОМИЯ,111,1,73.0,DIXY
1,МЯСНАЯ ГАСТРОНОМИЯ,111,2,96.0,DIXY
2,МЯСНАЯ ГАСТРОНОМИЯ,111,3,101.0,DIXY
3,МЯСНАЯ ГАСТРОНОМИЯ,111,4,122.0,DIXY
4,МЯСНАЯ ГАСТРОНОМИЯ,111,5,158.0,DIXY


In [41]:
quots_all=quots_all [['Plng', 'Plng_Size', 'QTA_RG', 'Формат']] 

quots_all['Plng_Size']=quots_all['Plng_Size'].astype(str)
quots_all['QTA_RG']=quots_all['QTA_RG'].fillna(0)
quots_all['QTA_RG']=round(quots_all['QTA_RG'])
quots_all['QTA_RG']=quots_all['QTA_RG'].astype(int)

quots_all = quots_all.rename(columns = {'Plng':'Планограмма','Plng_Size':'Размер','QTA_RG':'Квота'})
quots_all.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Планограмма  29 non-null     object
 1   Размер       29 non-null     object
 2   Квота        29 non-null     int32 
 3   Формат       29 non-null     object
dtypes: int32(1), object(3)
memory usage: 940.0+ bytes


In [42]:
quots_all

,Планограмма,Размер,Квота,Формат
0,МЯСНАЯ ГАСТРОНОМИЯ,1,73,DIXY
1,МЯСНАЯ ГАСТРОНОМИЯ,2,96,DIXY
2,МЯСНАЯ ГАСТРОНОМИЯ,3,101,DIXY
3,МЯСНАЯ ГАСТРОНОМИЯ,4,122,DIXY
4,МЯСНАЯ ГАСТРОНОМИЯ,5,158,DIXY
5,МЯСНАЯ ГАСТРОНОМИЯ,6,179,DIXY
6,МЯСНАЯ ГАСТРОНОМИЯ,7,210,DIXY
7,МЯСНАЯ ГАСТРОНОМИЯ,8,230,DIXY
8,МЯСНАЯ ГАСТРОНОМИЯ,9,240,DIXY
9,МЯСНАЯ ГАСТРОНОМИЯ,10,258,DIXY


In [43]:
quots_all['Формат'].unique()

array(['DIXY'], dtype=object)

In [44]:
def determine_site_format(Size):
    if Size in ['0','1','2','3','4','5','6','7','8','9','10','11','12','77']:
        return 'DIXY'
    elif Size in ['20','21']:
        return 'GO'
    elif Size == '50':
        return 'TOWER'
    else:
        return ''

In [45]:
quots_all['Формат'] = quots_all['Размер'].apply(determine_site_format)

In [46]:
quots_all['Формат'].unique()

array(['DIXY', 'GO', 'TOWER'], dtype=object)

**Проверка на изменение квот по сумме**<a class="anchor" id="selection_3_3"></a>

In [47]:
#МОЛ.ГОР ЙОГУРТЫ
quots_all_mgi = quots_all.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"') 

quots_all_mgi_group = quots_all_mgi.groupby(['Формат']).agg({'Квота':'sum'}).reset_index()
quots_mgi_group=quots_mgi.groupby(['Формат']).agg({'Count_Quots':'sum'}).reset_index()

quots_mgi_answer = pd.merge(quots_all_mgi_group, quots_mgi_group, on = ['Формат'], how = 'left')
quots_mgi_answer['R']=quots_mgi_answer['Квота']-quots_mgi_answer['Count_Quots']

quots_mgi_answer

,Формат,Квота,Count_Quots,R
0,DIXY,1135,866,269
1,GO,51,53,-2
2,TOWER,110,110,0


In [48]:
#МОЛ.ГОР ТРАДИЦ
quots_all_mgt = quots_all.query('Планограмма=="МОЛ.ГОР ТРАДИЦ"') 

quots_all_mgt_group = quots_all_mgt.groupby(['Формат']).agg({'Квота':'sum'}).reset_index()
quots_mgt_group=quots_mgt.groupby(['Формат']).agg({'Count_Quots':'sum'}).reset_index()

quots_mgt_answer = pd.merge(quots_all_mgt_group, quots_mgt_group, on = ['Формат'], how = 'left')
quots_mgt_answer['R']=quots_mgt_answer['Квота']-quots_mgt_answer['Count_Quots']

quots_mgt_answer

,Формат,Квота,Count_Quots,R
0,DIXY,669,507,162
1,GO,44,38,6
2,TOWER,40,40,0


In [49]:
#МЯСНАЯ ГАСТРОНОМИЯ
quots_all_mg = quots_all.query('Планограмма=="МЯСНАЯ ГАСТРОНОМИЯ"') 

quots_all_mg_group = quots_all_mg.groupby(['Формат']).agg({'Квота':'sum'}).reset_index()
quots_mg_group=quots_mg.groupby(['Формат']).agg({'Count_Quots':'sum'}).reset_index()

quots_mg_answer = pd.merge(quots_all_mg_group, quots_mg_group, on = ['Формат'], how = 'left')
quots_mg_answer['R']=quots_mg_answer['Квота']-quots_mg_answer['Count_Quots']

quots_mg_answer

,Формат,Квота,Count_Quots,R
0,DIXY,2588,2275,313
1,GO,56,56,0
2,TOWER,50,50,0


if quots_mgi_answer['R'].sum()!=0:    print('Хм.. Чего-то не так в плнг МОЛ.ГОР ЙОГУРТЫ')
else:    print('Расхождений нет в сумме по форматам по плнг МОЛ.ГОР ЙОГУРТЫ')

<div class="alert alert-success">
    <b>✅ Наполнение по магазинам для листа "ТТ с переквотой"</b> </div> <a class="anchor" id="chapter4"></a>

***По уровням:***

[вначало](#chapter0)

%%time
#SET NOCOUNT ON; 

query = """
with 

shops as (SELECT [SiteId] ,[SiteExtId] ,[level6_descr] 'format',[level5_descr] 'terr', [level4_descr] 'obl',[level3_descr] 'fo'  
		 FROM [DWH].[dbo].[DNodes]  --i on i.SiteID = n.SiteID
		 WHERE 1=1 
         and not level4_descr in ('СВЕРДЛОВСКАЯ ОБЛАСТЬ','ТЮМЕНСКАЯ ОБЛАСТЬ','ЧЕЛЯБИНСКАЯ ОБЛАСТЬ','АРХАНГЕЛЬСКАЯ ОБЛАСТЬ','ВОЛОГОДСКАЯ ОБЛАСТЬ','КАРЕЛИЯ','МУРМАНСКАЯ ОБЛАСТЬ','ПСКОВСКАЯ ОБЛАСТЬ') 
         and LegalEntityName like 'АО "ДИКСИ ЮГ"' and [ClosedDate] is NULL and level2_descr='ДИКСИ' ),
am as   (SELECT n.SiteId, n.SiteExtId, 
        case n.fo when 'ЦЕНТРАЛЬНЫЙ' Then 'ЦФО' when 'СЕВЕРО-ЗАПАДНЫЙ' Then 'СЗФО' when 'УРАЛЬСКИЙ' Then 'УФО' end 'FO', 
        n.format 'Format', n.obl 'Devision', n.terr 'Terr', atr.InOut,
        a.ItemId, atr.PlngDescription 'Plng', atr.Area, atr.PlngSize 'PLNG_SIZE', atr.PlngExtId  
		FROM dwh.gld.TypeSettingMatrixPeriod a with(nolock) 
        join dwh.gld.Attributes atr with(nolock) on atr.AttributeId = a.AttributeId 
        JOIN shops n with(nolock) on n.[SiteId] = a.SiteId 
		where 1=1 and cast(convert(varchar,getdate(),112)as int) between DateFrom and Dateto  
        and atr.[PlngExtId] in (122, 149) ), 
plng as (SELECT a.SiteId, a.SiteExtId, a.FO, a.Format, a.Devision, a.Terr, a.ItemId, a.Plng, a.Area, a.PLNG_SIZE, a.PlngExtId,
        case when (a.InOut = 'Временный' and p.AssortTypeName = 'РЕГУЛЯРНЫЙ') then 'Расширение' else a.InOut end 'InOut',
        LEFT(a.PLNG_SIZE,1) 'Claster' ,SUBSTRING(a.PLNG_SIZE,2,6) 'Size' ,  p.Level4_desc 'levels'
		FROM AM a with(nolock) JOIN dwh.dbo.DProducts p with(nolock) on a.ItemId = p.ItemId 
		where 1=1 and (AssortTypeName like '%РЕГУЛЯРНЫЙ%' or p.AssortTypeName like '%СЕЗОННЫЙ%') and a.InOut not like '%Расширение%')

SELECT 
     SiteExtId 'ТТ'
    ,case when Format='GO' then 'GO' when Format='TOWER' then 'TOWER' else 'DIXY' end 'Формат'
    ,Plng 'Планограмма'
    ,FO 'ФО'
    ,Devision 'ОБЛАСТЬ'
    ,case when levels in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else levels end 'Уровень'
    ,left(PLNG_SIZE,1) 'КЛАСТЕР'
    ,Size 'Размер'
    ,count(distinct ItemId) 'Наполнение'
    
FROM plng --where Format ='GO'
group by SiteExtId, FO, Devision, Format,Plng, PLNG_SIZE , Size, levels 

"""

gp=get_adw_connection()
site_count_item = pd.read_sql(query, con=gp)

gp.close()

sites_item_am=site_count_item 
sites_item_am['ТТ']=sites_item_am['ТТ'].astype(int)
sites_item_am['Размер']=sites_item_am['Размер'].astype(str) 
sites_item_am_mgi = sites_item_am.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"') 
sites_item_am_mgi.sample()

tt_mgi = pd.merge(formes_mgi_quots, sites_item_am_mgi, on = ['ФО', 'Планограмма', 'Уровень', 'ОБЛАСТЬ', 'КЛАСТЕР', 'Размер', 'Формат'], how = 'left')
tt_mgi['Отклонение']=tt_mgi['Квота']-tt_mgi['Наполнение']
tt_mgi['Отклонение']=tt_mgi['Отклонение'].fillna(0)
tt_mgi['Отклонение']=tt_mgi['Отклонение'].astype(int)
tt_mgi = tt_mgi.query('Отклонение<=0')
tt_mgi['Квота']=tt_mgi['Квота'].fillna(0)
tt_mgi['Квота']=tt_mgi['Квота'].astype(int)
#tt_mgi=tt_mgi[['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'Уровень', 'КЛАСТЕР', 'Размер', 'Квота', 'Наполнение', 'Отклонение', 'ТТ']]
#tt_mgi_perequtes = tt_mgi.sort_values(['ТТ', 'Формат', 'КЛАСТЕР', 'Размер'], ascending=[True, False, True, True])
tt_mgi_perequtes = tt_mgi.sort_values(['ТТ'], ascending=[True])
tt_mgi_perequtes=tt_mgi_perequtes[['ТТ', 'Формат', 'Планограмма', 'ФО', 'ОБЛАСТЬ', 'КЛАСТЕР', 'Размер', 'Квота', 'Наполнение', 'Отклонение']]

tt_mgi_perequtes.head(10) 

***Без уровней:***

In [50]:
%%time
#SET NOCOUNT ON; 

query = """


;with 

shops as (SELECT [SiteId] ,[SiteExtId] ,[level6_descr] 'format', [level4_descr] 'obl',[level3_descr] 'fo'  
		 FROM [DWH].[dbo].[DNodes]  --i on i.SiteID = n.SiteID
		 WHERE 1=1          --and not level4_descr in ('СВЕРДЛОВСКАЯ ОБЛАСТЬ','ТЮМЕНСКАЯ ОБЛАСТЬ','ЧЕЛЯБИНСКАЯ ОБЛАСТЬ','АРХАНГЕЛЬСКАЯ ОБЛАСТЬ','ВОЛОГОДСКАЯ ОБЛАСТЬ','КАРЕЛИЯ','МУРМАНСКАЯ ОБЛАСТЬ','ПСКОВСКАЯ ОБЛАСТЬ') 
		 and LegalEntityName like 'АО "ДИКСИ ЮГ"' and [ClosedDate] is NULL and level2_descr='ДИКСИ' ),
am as   (SELECT n.SiteId, n.SiteExtId, 
        case n.fo when 'ЦЕНТРАЛЬНЫЙ' Then 'ЦФО' when 'СЕВЕРО-ЗАПАДНЫЙ' Then 'СЗФО' when 'УРАЛЬСКИЙ' Then 'УФО' end 'FO', 
        n.format 'Format', n.obl 'Devision', atr.InOut,
        a.ItemId, atr.PlngDescription 'Plng', atr.Area, atr.PlngSize 'PLNG_SIZE', atr.PlngExtId  
		FROM dwh.gld.TypeSettingMatrixPeriod a with(nolock) 
        join dwh.gld.Attributes atr with(nolock) on atr.AttributeId = a.AttributeId 
        JOIN shops n with(nolock) on n.[SiteId] = a.SiteId 
		WHERE 1=1 
        and cast(convert(varchar,getdate(),112)as int) between DateFrom and Dateto  
        and atr.[PlngExtId] in (122, 149,111, 148) ), 
plng as (SELECT a.SiteId, a.SiteExtId, a.FO, a.Format, a.Devision, a.ItemId, a.Plng, a.Area, a.PLNG_SIZE, a.PlngExtId,
        case when (a.InOut = 'Временный' and p.AssortTypeName = 'РЕГУЛЯРНЫЙ') then 'Расширение' else a.InOut end 'InOut'
        ,SUBSTRING(a.PLNG_SIZE,2,6) 'Size' ,  p.Level4_desc 'levels'
		FROM AM a with(nolock) JOIN dwh.dbo.DProducts p with(nolock) on a.ItemId = p.ItemId 
		WHERE 1=1 
        and (AssortTypeName like '%РЕГУЛЯРНЫЙ%' or p.AssortTypeName like '%СЕЗОННЫЙ%') 
        and a.InOut not like '%Расширение%')

SELECT 
     SiteExtId 'ТТ'
    ,case when Format in ('M','S','L','XL') then 'GO' when Format='TOWER' then 'TOWER' --when Format = 'DARKSTORE' then 'DARKSTORE' 
     else 'DIXY' end 'Формат'
    ,Plng 'Планограмма'
    ,FO 'ФО'
    ,Devision 'ОБЛАСТЬ'
    ,left(PLNG_SIZE,1) 'КЛАСТЕР'
    ,Size 'Размер'
    ,count(distinct ItemId) 'Наполнение'

FROM plng 
group by SiteExtId, FO, Devision, Format,Plng, PLNG_SIZE , Size 

"""

gp=get_adw_connection()
site_count_itemm = pd.read_sql(query, con=gp)

gp.close()

CPU times: total: 422 ms
Wall time: 1min 11s


In [51]:
# наполнение пол планограммам по формату планограмме фо области кластеру размеру
display(site_count_itemm)

,ТТ,Формат,Планограмма,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Наполнение
0,32002.0,DIXY,МОЛ.ГОР ДЕТПИТ,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,M,1,42
1,32002.0,DIXY,МОЛ.ГОР ЙОГУРТЫ,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,T,1,114
2,32002.0,DIXY,МОЛ.ГОР ТРАДИЦ,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,T,1,79
3,32002.0,DIXY,МЯСНАЯ ГАСТРОНОМИЯ,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,T,6,116
4,32005.0,DIXY,МОЛ.ГОР ДЕТПИТ,ЦФО,БРЯНСКАЯ ОБЛАСТЬ,T,1,21
...,...,...,...,...,...,...,...,...
9657,97002.0,GO,МЯСНАЯ ГАСТРОНОМИЯ,ЦФО,МОСКВА,G,20,55
9658,97007.0,DIXY,МОЛ.ГОР ДЕТПИТ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,M,1,56
9659,97007.0,DIXY,МОЛ.ГОР ЙОГУРТЫ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,H,4,231
9660,97007.0,DIXY,МОЛ.ГОР ТРАДИЦ,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,H,4,130


In [52]:
sites_itemm_am=site_count_itemm 
sites_itemm_am['ТТ']=sites_itemm_am['ТТ'].astype(int)
sites_itemm_am['Размер']=sites_itemm_am['Размер'].astype(str) 
#sites_item_am = sites_item_am.rename(columns = {'Plng':'Планограмма', 'Count_sku':'Наполнение', 'Format':'Формат'})
#sites_item_am1=sites_item_am[['ТТ', 'Формат', 'Планограмма', 'ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР', 'Размер', 'Наполнение']]

In [53]:
#МОЛ.ГОР ЙОГУРТЫ

sites_itemm_am_mgi = sites_itemm_am.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"') 
tt_mgi_lev = pd.merge(sites_itemm_am_mgi, quots_all_mgi, on = ['Планограмма', 'Размер', 'Формат'], how = 'left')

tt_mgi_lev['Отклонение']=tt_mgi_lev['Квота']-tt_mgi_lev['Наполнение']
tt_mgi_lev['Отклонение']=tt_mgi_lev['Отклонение'].fillna(0)
tt_mgi_lev['Отклонение']=tt_mgi_lev['Отклонение'].astype(int)
tt_mgi_lev['Отклонение']=tt_mgi_lev['Квота']-tt_mgi_lev['Наполнение']
tt_mgi_lev['Отклонение']=tt_mgi_lev['Отклонение'].fillna(0)
tt_mgi_lev['Отклонение']=tt_mgi_lev['Отклонение'].astype(int)
#tt_mgi_lev = tt_mgi_lev.query('Отклонение<=0')

tt_mgi_lev['Квота']=tt_mgi_lev['Квота'].fillna(0)
tt_mgi_lev['Квота']=tt_mgi_lev['Квота'].astype(int)
tt_mgi_lev = tt_mgi_lev.sort_values(['ТТ'], ascending=[True])


tt_mgi_minus_lev = tt_mgi_lev.query('Отклонение<0')
tt_mgi_count_lev = str(len(tt_mgi_minus_lev['ТТ'].drop_duplicates()))
tt_mgi_sum_v= 'Магазинов с переквотой на сегодня = ' + tt_mgi_count_lev
print(tt_mgi_sum_v)

tt_mgi_null = tt_mgi_lev.query('Отклонение==0')
tt_mgi_count_n = str(len(tt_mgi_null['ТТ'].drop_duplicates()))
tt_mgi_sum_vn= 'Магазинов с критическим наполнением на сегодня = ' + tt_mgi_count_n
print(tt_mgi_sum_vn)

tt_mgi_lev.sample()

Магазинов с переквотой на сегодня = 14
Магазинов с критическим наполнением на сегодня = 48


,ТТ,Формат,Планограмма,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Наполнение,Квота,Отклонение
1441,77257,DIXY,МОЛ.ГОР ЙОГУРТЫ,ЦФО,МОСКВА,H,4,241,267,26


In [54]:
#МОЛ.ГОР ТРАДИЦ

sites_itemm_am_mgt = sites_itemm_am.query('Планограмма=="МОЛ.ГОР ТРАДИЦ"') 

tt_mgt_lev = pd.merge(sites_itemm_am_mgt, quots_all_mgt, on = ['Планограмма', 'Размер', 'Формат'], how = 'left')
tt_mgt_lev['Отклонение']=tt_mgt_lev['Квота']-tt_mgt_lev['Наполнение']
tt_mgt_lev['Отклонение']=tt_mgt_lev['Отклонение'].fillna(0)
tt_mgt_lev['Отклонение']=tt_mgt_lev['Отклонение'].astype(int)
#tt_mgt_lev = tt_mgt_lev.query('Отклонение<=0')
tt_mgt_lev['Квота']=tt_mgt_lev['Квота'].fillna(0)
tt_mgt_lev['Квота']=tt_mgt_lev['Квота'].astype(int)
tt_mgt_lev = tt_mgt_lev.sort_values(['ТТ'], ascending=[True])

tt_mgt_minus_lev = tt_mgt_lev.query('Отклонение<0')
tt_mgt_count_lev = str(len(tt_mgt_minus_lev['ТТ'].drop_duplicates()))
tt_mgt_sum_v= 'Магазинов с переквотой на сегодня = ' + tt_mgt_count_lev
print(tt_mgt_sum_v)

tt_mgt_null = tt_mgt_lev.query('Отклонение==0')
tt_mgt_count_n = str(len(tt_mgt_null['ТТ'].drop_duplicates()))
tt_mgt_sum_vn= 'Магазинов с критическим наполнением на сегодня = ' + tt_mgt_count_n
print(tt_mgt_sum_vn)

tt_mgt_lev.sample()

Магазинов с переквотой на сегодня = 8
Магазинов с критическим наполнением на сегодня = 50


,ТТ,Формат,Планограмма,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Наполнение,Квота,Отклонение
1244,76499,DIXY,МОЛ.ГОР ТРАДИЦ,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,M,4,110,162,52


In [55]:
#МЯСНАЯ ГАСТРОНОМИЯ

sites_itemm_am_mg = sites_itemm_am.query('Планограмма=="МЯСНАЯ ГАСТРОНОМИЯ"') 

tt_mg_lev = pd.merge(sites_itemm_am_mg, quots_all_mg, on = ['Планограмма', 'Размер', 'Формат'], how = 'left')
tt_mg_lev['Отклонение']=tt_mg_lev['Квота']-tt_mg_lev['Наполнение']
tt_mg_lev['Отклонение']=tt_mg_lev['Отклонение'].fillna(0)
tt_mg_lev['Отклонение']=tt_mg_lev['Отклонение'].astype(int)
#tt_mg_lev = tt_mg_lev.query('Отклонение<=0')
tt_mg_lev['Квота']=tt_mg_lev['Квота'].fillna(0)
tt_mg_lev['Квота']=tt_mg_lev['Квота'].astype(int)
tt_mg_lev = tt_mg_lev.sort_values(['ТТ'], ascending=[True])

tt_mg_minus_lev = tt_mg_lev.query('Отклонение<0')
tt_mg_count_lev = str(len(tt_mg_minus_lev['ТТ'].drop_duplicates()))
tt_mg_sum_v= 'Магазинов с переквотой на сегодня = ' + tt_mg_count_lev
print(tt_mg_sum_v)

tt_mg_null = tt_mg_lev.query('Отклонение==0')
tt_mg_count_n = str(len(tt_mg_null['ТТ'].drop_duplicates()))
tt_mg_sum_vn= 'Магазинов с критическим наполнением на сегодня = ' + tt_mg_count_n
print(tt_mg_sum_vn)

tt_mg_lev.sample()

Магазинов с переквотой на сегодня = 218
Магазинов с критическим наполнением на сегодня = 45


,ТТ,Формат,Планограмма,ФО,ОБЛАСТЬ,КЛАСТЕР,Размер,Наполнение,Квота,Отклонение
1167,71050,DIXY,МЯСНАЯ ГАСТРОНОМИЯ,ЦФО,ТУЛЬСКАЯ ОБЛАСТЬ,L,5,135,158,23


<div class="alert alert-success">
    <b>✅ Расчет доли ТТ</b> </div> 
    <a class="anchor" id="chapter5"></a>

[вначало](#chapter0)

In [56]:
%%time
#SET NOCOUNT ON; 

query = """

SELECT 
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end 'ФО'
,s.level4_descr 'ОБЛАСТЬ'
,case when [level6_descr] in ('M','S','L','XL')  then 'GO' --when [Level6_descr]='DARKSTORE' then 'DARKSTORE' 
when [level6_descr]='TOWER' then 'TOWER'  
 else 'DIXY' end 'Формат'
,PlngDescription 'Планограмма' 
--,PlngExtId 'NumPlng'
--,case when d.Level4_desc in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else d.Level4_desc end 'Уровень'
--,LEFT(PlngSize,1) 'КЛАСТЕР' 
,SUBSTRING(PlngSize,2,6) 'Размер'     
,count(distinct s.SiteExtId) 'ТТ_max'

FROM [DWH].[GLD].[TypeSettingMatrixPeriod] mp
JOIN [DWH].[GLD].[Attributes] a on mp.AttributeID = a.AttributeId
JOIN [DWH].[dbo].[DProducts] d on mp.ItemId = d.ItemId
JOIN [DWH].[dbo].[DNodes] s on mp.SiteId = s.SiteID
WHERE 1=1 and format(getdate(), 'yyyyMMdd') between mp.DateFrom and mp.DateTo
and PlngDescription in ('МОЛ.ГОР ЙОГУРТЫ', 'МОЛ.ГОР ТРАДИЦ', 'МЯСНАЯ ГАСТРОНОМИЯ') and s.[ClosedDate] is NULL 
group by 
s.level3_descr ,s.level4_descr ,case when [level6_descr] in ('M','S','L','XL') then 'GO' --when [Level6_descr]='DARKSTORE' then 'DARKSTORE' 
when [level6_descr]='TOWER' then 'TOWER' else 'DIXY' end 
,PlngDescription --,case when d.Level4_desc in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else d.Level4_desc end 
--,LEFT(PlngSize,1) 
,SUBSTRING(PlngSize,2,6)

"""

gp=get_adw_connection()
all_plngs = pd.read_sql(query, con=gp)
gp.close()

CPU times: total: 15.6 ms
Wall time: 9.91 s


In [57]:
all_plng = all_plngs[['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'Размер', 'ТТ_max']]
#all_plng.query('Формат!="DIXY"')

In [58]:
display(all_plng)
display(all_plng['Формат'].unique())
display(quots_all['Формат'].unique())

,ФО,ОБЛАСТЬ,Формат,Планограмма,Размер,ТТ_max
0,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,DIXY,МОЛ.ГОР ЙОГУРТЫ,4,77
1,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,DIXY,МОЛ.ГОР ТРАДИЦ,4,76
2,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,DIXY,МЯСНАЯ ГАСТРОНОМИЯ,2,2
3,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,DIXY,МЯСНАЯ ГАСТРОНОМИЯ,3,9
4,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,DIXY,МЯСНАЯ ГАСТРОНОМИЯ,5,9
...,...,...,...,...,...,...
281,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,DIXY,МОЛ.ГОР ЙОГУРТЫ,4,50
282,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,DIXY,МОЛ.ГОР ТРАДИЦ,2,4
283,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,DIXY,МОЛ.ГОР ТРАДИЦ,3,12
284,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,DIXY,МЯСНАЯ ГАСТРОНОМИЯ,12,13


array(['DIXY', 'GO', 'TOWER'], dtype=object)

array(['DIXY', 'GO', 'TOWER'], dtype=object)

**МОЛ.ГОР ЙОГУРТЫ**  <a class="anchor" id="selection_5_1"></a>

In [59]:
all_plng_mgi = all_plng.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"') 
all_plng_mgi['Размер']=all_plng_mgi['Размер'].astype(str)

formes_mgi_tt = pd.merge(quots_all.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"'), all_plng_mgi, on = ['Формат', 'Планограмма', 'Размер'], how = 'left')
formes_mgi_tt['ТТ_max']=formes_mgi_tt['ТТ_max'].fillna(0)
formes_mgi_tt['ТТ_max']=formes_mgi_tt['ТТ_max'].astype(int)

tt_group_mgi = tt_mgi_lev.query('Отклонение<=0').groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'Размер']).agg({'ТТ':'count'}).reset_index() # ТТ с переквотой

tt_plng_mgi = pd.merge(formes_mgi_tt, tt_group_mgi, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left') #2720 rows
tt_plng_mgi['ТТ']=tt_plng_mgi['ТТ'].fillna(0)
tt_plng_mgi['ТТ']=tt_plng_mgi['ТТ'].astype(int)

tt_plng_mgi['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgi['ТТ']*100/tt_plng_mgi['ТТ_max']
tt_plng_mgi['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgi['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_plng_mgi['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgi['Доля ТТ c полным наполнением или переквотой,%'].round(2)
#tt_plng_mgi.query('Формат=="TOWER"')

tt_plng_mgi['Размер']=tt_plng_mgi['Размер'].astype(str)
tt_plng_mgi['ТТ_max']=tt_plng_mgi['ТТ_max'].fillna(0)
tt_plng_mgi['ТТ_max']=tt_plng_mgi['ТТ_max'].astype(int)

tt_plng_mgi_s=tt_plng_mgi#.query('ТТ>0')
tt_plng_mgi_s.sample()

,Планограмма,Размер,Квота,Формат,ФО,ОБЛАСТЬ,ТТ_max,ТТ,"Доля ТТ c полным наполнением или переквотой,%"
17,МОЛ.ГОР ЙОГУРТЫ,2,192,DIXY,ЦФО,МОСКВА,61,0,0.0


In [60]:
tt_plng_mgi_s = tt_plng_mgi_s.rename(columns = {'ТТ':'Кол-во ТТ с полным наполнением или переквотой'})
tt_plng_mgi_s['Размер']=tt_plng_mgi_s['Размер'].astype(int)
tt_plng_mgi_s=tt_plng_mgi_s[['Формат', 'ФО', 'ОБЛАСТЬ', 'Размер', 'Доля ТТ c полным наполнением или переквотой,%', 'Кол-во ТТ с полным наполнением или переквотой']]

tt_plng_mgi_shablon=all_plng_mgi[['Формат', 'ФО', 'ОБЛАСТЬ', 'Размер']].drop_duplicates(inplace = False).sort_values(['Формат', 'Размер'])
tt_plng_mgi_shablon.sample(3)

,Формат,ФО,ОБЛАСТЬ,Размер
36,DIXY,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,4
90,DIXY,ЦФО,МОСКВА,2
6,GO,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,20


In [61]:
#tt_plng_mgi_shablon.query('Формат=="GO"')

In [62]:
#all_plng_mgi.sample() #всего магазинов в плнг
all_plng_mgim=all_plng_mgi[['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер', 'ТТ_max']]

tt_plng_mgi_s['Размер']=tt_plng_mgi_s['Размер'].astype(str)
tt_plng_mgi_sf=pd.merge(tt_plng_mgi_shablon, tt_plng_mgi_s, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left')

tt_plng_mgi_sf=pd.merge(tt_plng_mgi_sf, all_plng_mgim, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left')

tt_plng_mgi_sf['Кол-во ТТ с полным наполнением или переквотой']=tt_plng_mgi_sf['Кол-во ТТ с полным наполнением или переквотой'].fillna(0)
tt_plng_mgi_sf['Кол-во ТТ с полным наполнением или переквотой']=tt_plng_mgi_sf['Кол-во ТТ с полным наполнением или переквотой'].astype(int)
tt_plng_mgi_sf['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgi_sf['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_plng_mgi_sf['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgi_sf['Доля ТТ c полным наполнением или переквотой,%'].astype(int)
tt_plng_mgi_sf['Размер']=tt_plng_mgi_sf['Размер'].astype(int)


tt_mgi_d=tt_plng_mgi_sf.pivot_table( index=['Формат','ФО', 'ОБЛАСТЬ'], 
                                    values=['Доля ТТ c полным наполнением или переквотой,%', 'Кол-во ТТ с полным наполнением или переквотой', 'ТТ_max'],
                                columns='Размер', aggfunc='sum').reset_index()
#dolya_tt_mgi = tt_plng_mgi[['ФО', 'ОБЛАСТЬ', 'Размер', 'ТТ']].drop_duplicates()

tt_mgi_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mgi_d['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_mgi_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mgi_d['Доля ТТ c полным наполнением или переквотой,%'].round(2)
tt_mgi_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mgi_d['Доля ТТ c полным наполнением или переквотой,%'].astype(int)

tt_mgi_d['Кол-во ТТ с полным наполнением или переквотой']=tt_mgi_d['Кол-во ТТ с полным наполнением или переквотой'].fillna(0)
tt_mgi_d['Кол-во ТТ с полным наполнением или переквотой']=tt_mgi_d['Кол-во ТТ с полным наполнением или переквотой'].astype(int)

tt_mgi_d['ТТ_max']=tt_mgi_d['ТТ_max'].fillna(0)
tt_mgi_d['ТТ_max']=tt_mgi_d['ТТ_max'].astype(int)

tt_mgi_d=tt_mgi_d.sort_values(['Формат'], ascending=[False])
tt_mgi_d.sample(5)

Формат    ФО               ОБЛАСТЬ  \
Размер                                      
9        DIXY   ЦФО                МОСКВА   
5        DIXY   ЦФО  ВЛАДИМИРСКАЯ ОБЛАСТЬ   
8        DIXY   ЦФО   КОСТРОМСКАЯ ОБЛАСТЬ   
15       DIXY   ЦФО      ТУЛЬСКАЯ ОБЛАСТЬ   
1        DIXY  СЗФО  НОВГОРОДСКАЯ ОБЛАСТЬ   

       Доля ТТ c полным наполнением или переквотой,%                    ...  \
Размер                                             1  2  3  4 20 21 50  ...   
9                                                  0  0  0  0  0  0  0  ...   
5                                                  0  0  0  0  0  0  0  ...   
8                                                  0  0  0  0  0  0  0  ...   
15                                                 0  0  0  0  0  0  0  ...   
1                                                  0  0  0  0  0  0  0  ...   

       Кол-во ТТ с полным наполнением или переквотой    ТТ_max               \
Размер                                            50 77      1   2   3    4   
9                                                  0  0    139  61  54  267   
5                                                  0  0     19   2   3   30   
8                                                  0  0      1   1   1    5   
15                                                 0  0     17   4   9   46   
1                                                  0  0      8   1   2    9   

                    
Размер 20 21 50 77  
9       0  0  0  1  
5       0  0  0  0  
8       0  0  0  0  
15      0  0  0  0  
1       0  0  0  0  

[5 rows x 27 columns]

**МОЛ.ГОР ТРАДИЦ**  

In [63]:
all_plng_mgt = all_plng.query('Планограмма=="МОЛ.ГОР ТРАДИЦ"') 
all_plng_mgt['Размер']=all_plng_mgt['Размер'].astype(str)

formes_mgt_tt = pd.merge(quots_all.query('Планограмма=="МОЛ.ГОР ТРАДИЦ"'), all_plng_mgt, on = ['Формат', 'Планограмма', 'Размер'], how = 'left')
formes_mgt_tt['ТТ_max']=formes_mgt_tt['ТТ_max'].fillna(0)
formes_mgt_tt['ТТ_max']=formes_mgt_tt['ТТ_max'].astype(int)

tt_group_mgt = tt_mgt_lev.query('Отклонение<=0').groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'Размер']).agg({'ТТ':'count'}).reset_index() # ТТ с переквотой

tt_plng_mgt = pd.merge(formes_mgt_tt, tt_group_mgt, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left') #2720 rows
tt_plng_mgt['ТТ']=tt_plng_mgt['ТТ'].fillna(0)
tt_plng_mgt['ТТ']=tt_plng_mgt['ТТ'].astype(int)

tt_plng_mgt['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgt['ТТ']*100/tt_plng_mgt['ТТ_max']
tt_plng_mgt['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgt['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_plng_mgt['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgt['Доля ТТ c полным наполнением или переквотой,%'].round(2)
#tt_plng_mgt.query('Формат=="TOWER"')

tt_plng_mgt['Размер']=tt_plng_mgt['Размер'].astype(str)
tt_plng_mgt['ТТ_max']=tt_plng_mgt['ТТ_max'].fillna(0)
tt_plng_mgt['ТТ_max']=tt_plng_mgt['ТТ_max'].astype(int)

tt_plng_mgt_s=tt_plng_mgt.query('ТТ>0')
tt_plng_mgt_s.sample()

,Планограмма,Размер,Квота,Формат,ФО,ОБЛАСТЬ,ТТ_max,ТТ,"Доля ТТ c полным наполнением или переквотой,%"
35,МОЛ.ГОР ТРАДИЦ,3,135,DIXY,ЦФО,СМОЛЕНСКАЯ ОБЛАСТЬ,5,1,20.0


In [64]:
tt_plng_mgt_s = tt_plng_mgt_s.rename(columns = {'ТТ':'Кол-во ТТ с полным наполнением или переквотой'})
tt_plng_mgt_s['Размер']=tt_plng_mgt_s['Размер'].astype(int)
tt_plng_mgt_s=tt_plng_mgt_s[['Формат', 'ФО', 'ОБЛАСТЬ', 'Размер', 'Доля ТТ c полным наполнением или переквотой,%', 'Кол-во ТТ с полным наполнением или переквотой']]

tt_plng_mgt_shablon=all_plng_mgt[['Формат', 'ФО', 'ОБЛАСТЬ', 'Размер']].drop_duplicates(inplace = False).sort_values(['Формат', 'Размер'])
tt_plng_mgt_shablon.sample(3)

,Формат,ФО,ОБЛАСТЬ,Размер
238,DIXY,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,4
149,GO,ЦФО,МОСКОВСКАЯ ОБЛАСТЬ,20
87,DIXY,ЦФО,КОСТРОМСКАЯ ОБЛАСТЬ,3


In [65]:
#all_plng_mgt.sample() #всего магазинов в плнг
all_plng_mgtm=all_plng_mgt[['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер', 'ТТ_max']]

tt_plng_mgt_s['Размер']=tt_plng_mgt_s['Размер'].astype(str)
tt_plng_mgt_sf=pd.merge(tt_plng_mgt_shablon, tt_plng_mgt_s, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left')

tt_plng_mgt_sf=pd.merge(tt_plng_mgt_sf, all_plng_mgtm, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left')

tt_plng_mgt_sf['Кол-во ТТ с полным наполнением или переквотой']=tt_plng_mgt_sf['Кол-во ТТ с полным наполнением или переквотой'].fillna(0)
tt_plng_mgt_sf['Кол-во ТТ с полным наполнением или переквотой']=tt_plng_mgt_sf['Кол-во ТТ с полным наполнением или переквотой'].astype(int)
tt_plng_mgt_sf['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgt_sf['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_plng_mgt_sf['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mgt_sf['Доля ТТ c полным наполнением или переквотой,%'].astype(int)
tt_plng_mgt_sf['Размер']=tt_plng_mgt_sf['Размер'].astype(int)


tt_mgt_d=tt_plng_mgt_sf.pivot_table( index=['Формат','ФО', 'ОБЛАСТЬ'], 
                                    values=['Доля ТТ c полным наполнением или переквотой,%', 'Кол-во ТТ с полным наполнением или переквотой', 'ТТ_max'],
                                columns='Размер', aggfunc='sum').reset_index()
#dolya_tt_mgt = tt_plng_mgt[['ФО', 'ОБЛАСТЬ', 'Размер', 'ТТ']].drop_duplicates()

tt_mgt_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mgt_d['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_mgt_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mgt_d['Доля ТТ c полным наполнением или переквотой,%'].round(2)
tt_mgt_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mgt_d['Доля ТТ c полным наполнением или переквотой,%'].astype(int)

tt_mgt_d['Кол-во ТТ с полным наполнением или переквотой']=tt_mgt_d['Кол-во ТТ с полным наполнением или переквотой'].fillna(0)
tt_mgt_d['Кол-во ТТ с полным наполнением или переквотой']=tt_mgt_d['Кол-во ТТ с полным наполнением или переквотой'].astype(int)

tt_mgt_d['ТТ_max']=tt_mgt_d['ТТ_max'].fillna(0)
tt_mgt_d['ТТ_max']=tt_mgt_d['ТТ_max'].astype(int)

tt_mgt_d=tt_mgt_d.sort_values(['Формат'], ascending=[False])
tt_mgt_d.sample(5)

Формат    ФО             ОБЛАСТЬ  \
Размер                                    
13       DIXY   ЦФО  СМОЛЕНСКАЯ ОБЛАСТЬ   
4        DIXY   ЦФО    БРЯНСКАЯ ОБЛАСТЬ   
3        DIXY  СЗФО     САНКТ-ПЕТЕРБУРГ   
6        DIXY   ЦФО  ИВАНОВСКАЯ ОБЛАСТЬ   
10       DIXY   ЦФО  МОСКОВСКАЯ ОБЛАСТЬ   

       Доля ТТ c полным наполнением или переквотой,%                      ...  \
Размер                                             1   2   3  4 20 21 50  ...   
13                                                 0  50  20  0  0  0  0  ...   
4                                                  0   0   0  0  0  0  0  ...   
3                                                 13   0   0  0  0  0  0  ...   
6                                                  0   0   0  0  0  0  0  ...   
10                                                 0   0   0  0  0  0  0  ...   

       Кол-во ТТ с полным наполнением или переквотой    ТТ_max               \
Размер                                            50 77      1   2   3    4   
13                                                 0  0     11   2   5   14   
4                                                  0  0      9   0   2   18   
3                                                  0  0    106  44  40  128   
6                                                  0  0      5   1   0    7   
10                                                 0  0    244  86  97  491   

                    
Размер 20 21 50 77  
13      0  0  0  0  
4       0  0  0  0  
3       0  0  0  0  
6       0  0  0  0  
10      0  0  0  0  

[5 rows x 27 columns]

**МЯСНАЯ ГАСТРОНОМИЯ**  

In [66]:
all_plng_mg = all_plng.query('Планограмма=="МЯСНАЯ ГАСТРОНОМИЯ"') 
all_plng_mg['Размер']=all_plng_mg['Размер'].astype(str)

formes_mg_tt = pd.merge(quots_all.query('Планограмма=="МЯСНАЯ ГАСТРОНОМИЯ"'), all_plng_mg, on = ['Формат', 'Планограмма', 'Размер'], how = 'left')
formes_mg_tt['ТТ_max']=formes_mg_tt['ТТ_max'].fillna(0)
formes_mg_tt['ТТ_max']=formes_mg_tt['ТТ_max'].astype(int)

tt_group_mg = tt_mg_lev.groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'Размер']).agg({'ТТ':'count'}).reset_index() # ТТ с переквотой

tt_plng_mg = pd.merge(formes_mg_tt, tt_group_mg, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left') #2720 rows
tt_plng_mg['ТТ']=tt_plng_mg['ТТ'].fillna(0)
tt_plng_mg['ТТ']=tt_plng_mg['ТТ'].astype(int)

tt_plng_mg['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mg['ТТ']*100/tt_plng_mg['ТТ_max']
tt_plng_mg['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mg['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_plng_mg['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mg['Доля ТТ c полным наполнением или переквотой,%'].round(2)
#tt_plng_mg.query('Формат=="TOWER"')

tt_plng_mg['Размер']=tt_plng_mg['Размер'].astype(str)
tt_plng_mg['ТТ_max']=tt_plng_mg['ТТ_max'].fillna(0)
tt_plng_mg['ТТ_max']=tt_plng_mg['ТТ_max'].astype(int)

tt_plng_mg_s=tt_plng_mg.query('ТТ>0')
tt_plng_mg_s.sample()

,Планограмма,Размер,Квота,Формат,ФО,ОБЛАСТЬ,ТТ_max,ТТ,"Доля ТТ c полным наполнением или переквотой,%"
51,МЯСНАЯ ГАСТРОНОМИЯ,6,179,DIXY,ЦФО,ОРЛОВСКАЯ ОБЛАСТЬ,1,1,100.0


In [67]:
tt_plng_mg_s = tt_plng_mg_s.rename(columns = {'ТТ':'Кол-во ТТ с полным наполнением или переквотой'})
tt_plng_mg_s['Размер']=tt_plng_mg_s['Размер'].astype(int)
tt_plng_mg_s=tt_plng_mg_s[['Формат', 'ФО', 'ОБЛАСТЬ', 'Размер', 'Доля ТТ c полным наполнением или переквотой,%', 'Кол-во ТТ с полным наполнением или переквотой']]

tt_plng_mg_shablon=all_plng_mg[['Формат', 'ФО', 'ОБЛАСТЬ', 'Размер']].drop_duplicates(inplace = False).sort_values(['Формат', 'Размер'])
tt_plng_mg_shablon.sample(3)

,Формат,ФО,ОБЛАСТЬ,Размер
243,DIXY,ЦФО,СМОЛЕНСКАЯ ОБЛАСТЬ,12
259,DIXY,ЦФО,ЯРОСЛАВСКАЯ ОБЛАСТЬ,8
68,DIXY,СЗФО,САНКТ-ПЕТЕРБУРГ,6


In [68]:
#all_plng_mg.sample() #всего магазинов в плнг
all_plng_mgm=all_plng_mg[['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер', 'ТТ_max']]

tt_plng_mg_s['Размер']=tt_plng_mg_s['Размер'].astype(str)
tt_plng_mg_sf=pd.merge(tt_plng_mg_shablon, tt_plng_mg_s, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left')

tt_plng_mg_sf=pd.merge(tt_plng_mg_sf, all_plng_mgm, on = ['ФО', 'ОБЛАСТЬ', 'Формат',  'Размер'], how = 'left')

tt_plng_mg_sf['Кол-во ТТ с полным наполнением или переквотой']=tt_plng_mg_sf['Кол-во ТТ с полным наполнением или переквотой'].fillna(0)
tt_plng_mg_sf['Кол-во ТТ с полным наполнением или переквотой']=tt_plng_mg_sf['Кол-во ТТ с полным наполнением или переквотой'].astype(int)
tt_plng_mg_sf['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mg_sf['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_plng_mg_sf['Доля ТТ c полным наполнением или переквотой,%']=tt_plng_mg_sf['Доля ТТ c полным наполнением или переквотой,%'].astype(int)
tt_plng_mg_sf['Размер']=tt_plng_mg_sf['Размер'].astype(int)

tt_mg_d=tt_plng_mg_sf.pivot_table( index=['Формат','ФО', 'ОБЛАСТЬ'], 
                                    values=['Доля ТТ c полным наполнением или переквотой,%', 'Кол-во ТТ с полным наполнением или переквотой', 'ТТ_max'],
                                columns='Размер', aggfunc='sum').reset_index()
#dolya_tt_mg = tt_plng_mg[['ФО', 'ОБЛАСТЬ', 'Размер', 'ТТ']].drop_duplicates()

tt_mg_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mg_d['Доля ТТ c полным наполнением или переквотой,%'].fillna(0)
tt_mg_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mg_d['Доля ТТ c полным наполнением или переквотой,%'].round(2)
tt_mg_d['Доля ТТ c полным наполнением или переквотой,%']=tt_mg_d['Доля ТТ c полным наполнением или переквотой,%'].astype(int)

tt_mg_d['Кол-во ТТ с полным наполнением или переквотой']=tt_mg_d['Кол-во ТТ с полным наполнением или переквотой'].fillna(0)
tt_mg_d['Кол-во ТТ с полным наполнением или переквотой']=tt_mg_d['Кол-во ТТ с полным наполнением или переквотой'].astype(int)

tt_mg_d['ТТ_max']=tt_mg_d['ТТ_max'].fillna(0)
tt_mg_d['ТТ_max']=tt_mg_d['ТТ_max'].astype(int)

tt_mg_d=tt_mg_d.sort_values(['Формат'], ascending=[False])
tt_mg_d.sample(5)

Формат   ФО             ОБЛАСТЬ  \
Размер                                   
10       DIXY  ЦФО  МОСКОВСКАЯ ОБЛАСТЬ   
7        DIXY  ЦФО   КАЛУЖСКАЯ ОБЛАСТЬ   
9        DIXY  ЦФО              МОСКВА   
4        DIXY  ЦФО    БРЯНСКАЯ ОБЛАСТЬ   
15       DIXY  ЦФО    ТУЛЬСКАЯ ОБЛАСТЬ   

       Доля ТТ c полным наполнением или переквотой,%                           \
Размер                                             1    2    3    4    5    6   
10                                               100  100  100  100  100  100   
7                                                  0    0    0  100  100  100   
9                                                100  100  100  100  100   95   
4                                                  0    0    0    0  100  100   
15                                                 0    0  100  100  100  100   

             ... ТТ_max                                    
Размер    7  ...      7    8   9  10  11   12 20 21 50 77  
10      100  ...    237  163  53  72  38  174  0  0  0  0  
7       100  ...     14   14   2   5   1    8  0  0  0  0  
9       100  ...    130  117  24  49  16   89  0  0  0  1  
4       100  ...      9    4   7   3   0    0  0  0  0  0  
15      100  ...     31   17   3   6   6    6  0  0  0  0  

[5 rows x 51 columns]

<div class="alert alert-success">
    <b>✅ Расчет абсолютных переквот</b></div> </b></div> <a class="anchor" id="chapter7"></a>

[вначало](#chapter0)

In [69]:
%%time

query = """

with 
shops as
    (SELECT [SiteId] ,[SiteExtId] ,[level6_descr] 'format',[level5_descr] 'terr', [level4_descr] 'obl',[level3_descr] 'fo'  
    FROM [DWH].[dbo].[DNodes]  --i on i.SiteID = n.SiteID
    WHERE 1=1  and LegalEntityName like 'АО "ДИКСИ ЮГ"' and [ClosedDate] is NULL and level2_descr='ДИКСИ' ),

am as
    (SELECT n.SiteId, 
    n.SiteExtId, 
    case n.fo when 'ЦЕНТРАЛЬНЫЙ' Then 'ЦФО' when 'СЕВЕРО-ЗАПАДНЫЙ' Then 'СЗФО' when 'УРАЛЬСКИЙ' Then 'УФО' end 'FO', 
    n.format 'Format', 
    n.obl 'Devision', 
    atr.InOut,
    a.ItemId, atr.PlngDescription 'Plng', atr.Area, atr.PlngSize 'PLNG_SIZE', atr.PlngExtId  
    FROM dwh.gld.TypeSettingMatrixPeriod a with(nolock) 
	join dwh.gld.Attributes atr with(nolock) on atr.AttributeId = a.AttributeId 
	JOIN shops n with(nolock) on n.[SiteId] = a.SiteId 
	where 1=1 and cast(convert(varchar,getdate(),112)as int) between DateFrom and Dateto  
    and atr.[PlngExtId] in (122,  149, 148, 111 ) ), 
	
plng as
	(SELECT a.SiteId, a.SiteExtId, a.FO, a.Format, a.Devision, a.ItemId, a.Plng, a.Area, a.PLNG_SIZE, a.PlngExtId,
	case when (a.InOut = 'Временный' and p.AssortTypeName = 'РЕГУЛЯРНЫЙ') then 'Расширение' else a.InOut end 'InOut',
	LEFT(a.PLNG_SIZE,1) 'Claster'
    ,SUBSTRING(a.PLNG_SIZE,2,6) 'Размер'   
	, p.Level4_desc 'levels'
    
	FROM AM a with(nolock) 	
	JOIN dwh.dbo.DProducts p with(nolock) on a.ItemId = p.ItemId 
	where 1=1  and (AssortTypeName like '%РЕГУЛЯРНЫЙ%' or p.AssortTypeName like '%СЕЗОННЫЙ%') and a.InOut not like '%Расширение%')

SELECT 
      FO 'ФО'
	, Devision 'ОБЛАСТЬ'
	, case when Format in ('M','S','L','XL') then 'GO' when Format='TOWER' then 'TOWER' when Format = 'DARKSTORE' then 'DARKSTORE' else 'DIXY' end 'Формат' 
	, Plng 'Планограмма'
	, Claster 'КЛАСТЕР'
	, Размер 
	,  case when levels in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else levels end 'Уровень'
	, count(distinct ItemId) 'Наполнение'
FROM plng 
group by 
    FO, Devision, case when Format in ('M','S','L','XL') then 'GO' when Format='TOWER' then 'TOWER' when Format = 'DARKSTORE' then 'DARKSTORE'
    else 'DIXY' end, Plng, PLNG_SIZE , Claster, Размер, 
    case when levels in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else levels end

"""

gp=get_adw_connection()
count_items = pd.read_sql(query, con=gp)

gp.close()

CPU times: total: 203 ms
Wall time: 1min 3s


In [70]:
count_item=count_items
#count_item = count_item[['FO', 'Devision', 'Format', 'Plng', 'Claster', 'Size', 'lev', 'Count_sku']]
#count_item = count_item.rename(columns = {'FO':'ФО', 'Devision':'ОБЛАСТЬ', 'Format':'Формат', 'Plng':'Планограмма', 'Claster':'КЛАСТЕР', 'Size':'Размер', 'lev':'Уровень', 'Count_sku':'Наполнение'})
count_item.head(2)
#count_item.info()

,ФО,ОБЛАСТЬ,Формат,Планограмма,КЛАСТЕР,Размер,Уровень,Наполнение
0,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,DIXY,МОЛ.ГОР ДЕТПИТ,H,1,ДЕТСКОЕ МОЛОЧНОЕ ДОШКОЛЬНОЕ ПИТАНИЕ,55
1,СЗФО,ЛЕНИНГРАДСКАЯ ОБЛАСТЬ,DIXY,МОЛ.ГОР ДЕТПИТ,L,1,ДЕТСКОЕ МОЛОЧНОЕ ДОШКОЛЬНОЕ ПИТАНИЕ,33


In [71]:
def foo(row):
    if row.Наполнение==0:
        return 0
    else:
        return row.Откл

In [72]:
#МОЛ.ГОР ЙОГУРТЫ

count_item_mgi = count_item.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"') 
mgi_tt_otkl = pd.merge(count_item_mgi, quots_mgi, on = ['Планограмма', 'Уровень', 'Размер', 'Формат'], how = 'left') 
mgi_tt_otkl['Наполнение']=mgi_tt_otkl['Наполнение'].fillna(0)
mgi_tt_otkl['Откл']=mgi_tt_otkl['Count_Quots']-mgi_tt_otkl['Наполнение']
mgi_tt_otkl.head(2)

mgi_tt_otkl['Отклонение'] = mgi_tt_otkl.apply(foo, axis=1)
#mgi_tt_otkl.query('Наполнение>0')

mgi_tt_otkl = mgi_tt_otkl[['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'Уровень', 'КЛАСТЕР' , 'Размер', 'Отклонение', 'Наполнение' , 'Count_Quots']]

#mgi_tt_otkl.sample() 


#МОЛ.ГОР ТРАДИЦ

count_item_mgt = count_item.query('Планограмма=="МОЛ.ГОР ТРАДИЦ"') 
mgt_tt_otkl = pd.merge(count_item_mgt, quots_mgt, on = ['Планограмма', 'Уровень', 'Размер', 'Формат'], how = 'left') 
mgt_tt_otkl['Наполнение']=mgt_tt_otkl['Наполнение'].fillna(0)
mgt_tt_otkl['Откл']=mgt_tt_otkl['Count_Quots']-mgt_tt_otkl['Наполнение']
mgt_tt_otkl.head(2)

mgt_tt_otkl['Отклонение'] = mgt_tt_otkl.apply(foo, axis=1)
#mgt_tt_otkl.query('Наполнение>0')

mgt_tt_otkl = mgt_tt_otkl[['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'Уровень', 'КЛАСТЕР' , 'Размер', 'Отклонение', 'Наполнение' , 'Count_Quots']]

#mgt_tt_otkl.sample() 


#МЯСНАЯ ГАСТРОНОМИЯ

count_item_mg = count_item.query('Планограмма=="МЯСНАЯ ГАСТРОНОМИЯ"') 
mg_tt_otkl = pd.merge(count_item_mg, quots_mg, on = ['Планограмма', 'Уровень', 'Размер', 'Формат'], how = 'left') 
mg_tt_otkl['Наполнение']=mg_tt_otkl['Наполнение'].fillna(0)
mg_tt_otkl['Откл']=mg_tt_otkl['Count_Quots']-mg_tt_otkl['Наполнение']
mg_tt_otkl.head(2)

mg_tt_otkl['Отклонение'] = mg_tt_otkl.apply(foo, axis=1)
#mg_tt_otkl.query('Наполнение>0')

mg_tt_otkl = mg_tt_otkl[['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'Уровень', 'КЛАСТЕР' , 'Размер', 'Отклонение', 'Наполнение' , 'Count_Quots']]

#mg_tt_otkl.sample() 

<div class="alert alert-success">
    <b>✅ Расчет переквот по взвесу</b></div> </b></div> <a class="anchor" id="chapter8"></a>

[вначало](#chapter0)

In [73]:
%%time

query = """

with 
shops as
    (SELECT [SiteId] ,[SiteExtId] ,[level6_descr] 'format',[level5_descr] 'terr', [level4_descr] 'obl',[level3_descr] 'fo'  
    FROM [DWH].[dbo].[DNodes]  --i on i.SiteID = n.SiteID
    WHERE 1=1 and LegalEntityName like 'АО "ДИКСИ ЮГ"' and [ClosedDate] is NULL and level2_descr='ДИКСИ' ),

am as
    (SELECT n.SiteId, 
    n.SiteExtId, 
    case n.fo when 'ЦЕНТРАЛЬНЫЙ' Then 'ЦФО' when 'СЕВЕРО-ЗАПАДНЫЙ' Then 'СЗФО' when 'УРАЛЬСКИЙ' Then 'УФО' end 'FO', 
    n.format 'Format', 
    n.obl 'Devision', 
    n.terr 'Terr', 
    atr.InOut,
    a.ItemId, atr.PlngDescription 'Plng', atr.Area, atr.PlngSize 'PLNG_SIZE', atr.PlngExtId  
    
    FROM dwh.gld.TypeSettingMatrixPeriod a with(nolock) 
    join dwh.gld.Attributes atr with(nolock) on atr.AttributeId = a.AttributeId 
    JOIN shops n with(nolock) on n.[SiteId] = a.SiteId 
    
    where 1=1 and cast(convert(varchar,getdate(),112)as int) between DateFrom and Dateto  
    and atr.[PlngExtId] in (122,  149, 148, 111 ) ), 

plng as
    (SELECT a.SiteId, a.SiteExtId, a.FO, a.Format, a.Devision, a.Terr, a.ItemId, 
    a.Plng, a.Area, a.PLNG_SIZE, a.PlngExtId,
    case when (a.InOut = 'Временный' and p.AssortTypeName = 'РЕГУЛЯРНЫЙ') then 'Расширение' else a.InOut end 'InOut',
    LEFT(a.PLNG_SIZE,1) 'Claster'
   ,SUBSTRING(a.PLNG_SIZE,2,6) 'Size', p.Level4_desc 'levels'
    FROM AM a with(nolock) 
    JOIN dwh.dbo.DProducts p with(nolock) on a.ItemId = p.ItemId 
    where 1=1  and (AssortTypeName like '%РЕГУЛЯРНЫЙ%' or p.AssortTypeName like '%СЕЗОННЫЙ%') and a.InOut not like '%Расширение%')

SELECT 
    ItemId, FO 'ФО', Devision  'ОБЛАСТЬ', 
    case when Format in ('M','S','L','XL') then 'GO' when Format='TOWER' then 'TOWER' when Format = 'DARKSTORE' then 'DARKSTORE' else 'DIXY' end  'Формат'  , 
    Plng 'Планограмма', Claster 'КЛАСТЕР', Size'Размер' ,  
    case when levels in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else levels end 'Уровень', 
    count(distinct SiteExtID) 'Count_TT'
FROM plng 
group by 
    ItemId, FO, Devision, 
    case when Format in ('M','S','L','XL') then 'GO' when Format='TOWER' then 'TOWER' when Format = 'DARKSTORE' then 'DARKSTORE'
    else 'DIXY' end, Plng , PLNG_SIZE , Claster, Size, 
    case when levels in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else levels end

"""

gp=get_adw_connection()
now = pd.read_sql(query, con=gp)
gp.close()

CPU times: total: 3.19 s
Wall time: 1min 10s


In [74]:
now_items=now 
now_items = now_items.rename(columns = {'Count_TT':'КолТТ_присутствия'})
now_items.sample(3)

,ItemId,ФО,ОБЛАСТЬ,Формат,Планограмма,КЛАСТЕР,Размер,Уровень,КолТТ_присутствия
89562,939688,ЦФО,МОСКВА,DIXY,МОЛ.ГОР ЙОГУРТЫ,H,4,ПИТЬЕВЫЕ ЙОГУРТЫ И ДЕСЕРТЫ,194
53898,916546,ЦФО,СМОЛЕНСКАЯ ОБЛАСТЬ,DIXY,МЯСНАЯ ГАСТРОНОМИЯ,M,10,"СОСИСКИ, САРДЕЛЬКИ",2
79312,936857,ЦФО,МОСКВА,DIXY,МЯСНАЯ ГАСТРОНОМИЯ,H,11,ДЕЛИКАТЕСЫ,11


In [75]:
#now_items.query('Формат=="GO"')

In [76]:
%%time
#SET NOCOUNT ON; 

query = """

SELECT 
case when s.level3_descr = 'СЕВЕРО-ЗАПАДНЫЙ' then 'СЗФО' else 'ЦФО' end 'ФО'
,s.level4_descr 'ОБЛАСТЬ'
,case when [level6_descr] in ('M','S','L','XL')  then 'GO' when [level6_descr]='TOWER' then 'TOWER' when [Level6_descr] = 'DARKSTORE' then 'DARKSTORE' else 'DIXY' end 'Формат'
,PlngDescription 'Планограмма' 
--,PlngExtId 'NumPlng'
,case when d.Level4_desc in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else d.Level4_desc end 'Уровень'
,LEFT(PlngSize,1) 'КЛАСТЕР' 
,SUBSTRING(PlngSize,2,6) 'Размер'     
,count(distinct s.SiteExtId) 'ТТ_max'

FROM [DWH].[GLD].[TypeSettingMatrixPeriod] mp
JOIN [DWH].[GLD].[Attributes] a on mp.AttributeID = a.AttributeId
JOIN [DWH].[dbo].[DProducts] d on mp.ItemId = d.ItemId
JOIN [DWH].[dbo].[DNodes] s on mp.SiteId = s.SiteID
WHERE 1=1 and format(getdate(), 'yyyyMMdd') between mp.DateFrom and mp.DateTo
and PlngDescription in ('МОЛ.ГОР ЙОГУРТЫ', 'МОЛ.ГОР ТРАДИЦ', 'МЯСНАЯ ГАСТРОНОМИЯ') and s.[ClosedDate] is NULL 
group by 
s.level3_descr ,s.level4_descr ,case when [level6_descr] in ('M','S','L','XL')  then 'GO' when [level6_descr]='TOWER' then 'TOWER' when [Level6_descr] = 'DARKSTORE' then 'DARKSTORE' else 'DIXY' end 
,PlngDescription ,case when d.Level4_desc in ('МОЛОКО', 'КРЕМ И СЛИВКИ') then 'МОЛОКО ПАСТЕРИЗОВАННОЕ, СЛИВКИ ПАСТЕР.' else d.Level4_desc end 
,LEFT(PlngSize,1), SUBSTRING(PlngSize,2,6)

"""

gp=get_adw_connection()
all_plngsss = pd.read_sql(query, con=gp)
gp.close()

CPU times: total: 78.1 ms
Wall time: 12 s


In [77]:
#МОЛ.ГОР ЙОГУРТЫ
all_plngsss_mgi = all_plngsss.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"') 
all_plngsss_mgi.sample(3)

now_items_mgi = now_items.query('Планограмма=="МОЛ.ГОР ЙОГУРТЫ"')  

#Посчитаем наполнение по взвесу:
now_plng_mgi = pd.merge(now_items_mgi, all_plngsss_mgi, on=['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень'], how='left')
now_plng_mgi['p_TT'] = now_plng_mgi['КолТТ_присутствия'] / now_plng_mgi['ТТ_max']

now_plng_mgi_s = now_plng_mgi.groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень']).agg('p_TT').sum().reset_index()

#Добавляем взесы к табл с квотами:
df_mgi = pd.merge(mgi_tt_otkl, now_plng_mgi_s, on=['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень'], how='left') #2720 rows


df_mgi['p_TT']=df_mgi['p_TT'].fillna(0)
df_mgi['Взвес'] = df_mgi['Count_Quots']-df_mgi['p_TT']
df_mgi['Count_Quots']=df_mgi['Count_Quots'].fillna(0)

df_mgi['p_TT']=df_mgi['p_TT'].fillna(0)
df_mgi['Взвес'] = df_mgi['Count_Quots']-df_mgi['p_TT']

def fo(row):
    if row.p_TT==0:
        return 0
    else:
        return row.Взвес

df_mgi['ОтклПоВзвесу'] = df_mgi.apply(fo, axis=1)
df_mgi = df_mgi.rename(columns = {'Отклонение':'Откл.', 'ОтклПоВзвесу':'ПоВзвесу'})

df_mgi=df_mgi[['ФО', 'ОБЛАСТЬ', 'Формат','Планограмма','Уровень', 'КЛАСТЕР', 'Размер', 'Откл.', 'ПоВзвесу', 'Count_Quots']]
df_mgi['ПоВзвесу']=df_mgi['ПоВзвесу'].round(1)

#df_mgi['Доля ТТ с полным наполнением или переквотой,%']=df_mgi['Доля ТТ c полным наполнением или переквотой,%'].round(0)
df_mgi['Разница взвеса от откл.']=df_mgi['ПоВзвесу']-df_mgi['Откл.']

df_mgi = df_mgi.rename(columns = {'Count_Quots':'Квоты'})

df_mgi['Квоты']=df_mgi['Квоты'].fillna(0)
df_mgi['Откл.']=df_mgi['Откл.'].fillna(0)
df_mgi['ПоВзвесу']=df_mgi['ПоВзвесу'].fillna(0)
df_mgi['Разница взвеса от откл.']=df_mgi['Разница взвеса от откл.'].fillna(0)

#Формируем таблицу для выгрузки по формату DIXY
df_mgi_dixy = df_mgi.query('Формат=="DIXY"') 
pivot_mgi_dixy=df_mgi_dixy.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mgi_dixy['Квоты']=pivot_mgi_dixy['Квоты'].fillna(0)
pivot_mgi_dixy['Откл.']=pivot_mgi_dixy['Откл.'].fillna(0)
pivot_mgi_dixy['ПоВзвесу']=pivot_mgi_dixy['ПоВзвесу'].fillna(0)
pivot_mgi_dixy['Разница взвеса от откл.']=pivot_mgi_dixy['Разница взвеса от откл.'].fillna(0)
pivot_mgi_dixy['ПоВзвесу']=pivot_mgi_dixy['ПоВзвесу'].round(0)

#Формируем таблицу для выгрузки по формату GO
df_mgi_go = df_mgi.query('Формат=="GO"') 
pivot_mgi_go=df_mgi_go.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mgi_go['Квоты']=pivot_mgi_go['Квоты'].fillna(0)
pivot_mgi_go['Откл.']=pivot_mgi_go['Откл.'].fillna(0)
pivot_mgi_go['ПоВзвесу']=pivot_mgi_go['ПоВзвесу'].fillna(0)
pivot_mgi_go['Разница взвеса от откл.']=pivot_mgi_go['Разница взвеса от откл.'].fillna(0)
pivot_mgi_go['ПоВзвесу']=pivot_mgi_go['ПоВзвесу'].round(0)

#Формируем таблицу для выгрузки по формату TOWER
df_mgi_t = df_mgi.query('Формат=="TOWER"') 
pivot_mgi_t=df_mgi_t.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mgi_t['Квоты']=pivot_mgi_t['Квоты'].fillna(0)
pivot_mgi_t['Откл.']=pivot_mgi_t['Откл.'].fillna(0)
pivot_mgi_t['ПоВзвесу']=pivot_mgi_t['ПоВзвесу'].fillna(0)
pivot_mgi_t['Разница взвеса от откл.']=pivot_mgi_t['Разница взвеса от откл.'].fillna(0)
pivot_mgi_t['ПоВзвесу']=pivot_mgi_t['ПоВзвесу'].round(0)

#pivot_mgi_dixy.sample()
#pivot_mgi_go
#pivot_mgi_t

In [78]:
count_item_mgi.query('Уровень=="ДЕТСКОЕ МОЛОЧНОЕ ДОШКОЛЬНОЕ ПИТАНИЕ"')

#mgi_tt_otkl, now_plng_mgi_s
#count_item_mgi, quots_mgi

,ФО,ОБЛАСТЬ,Формат,Планограмма,КЛАСТЕР,Размер,Уровень,Наполнение


In [79]:
#МОЛ.ГОР ТРАДИЦ

all_plngsss_mgt = all_plngsss.query('Планограмма=="МОЛ.ГОР ТРАДИЦ"') 
all_plngsss_mgt.sample(3)

now_items_mgt = now_items.query('Планограмма=="МОЛ.ГОР ТРАДИЦ"')  

#Посчитаем наполнение по взвесу:
now_plng_mgt = pd.merge(now_items_mgt, all_plngsss_mgt, on=['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень'], how='left')
now_plng_mgt['p_TT'] = now_plng_mgt['КолТТ_присутствия'] / now_plng_mgt['ТТ_max']

now_plng_mgt_s = now_plng_mgt.groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень']).agg('p_TT').sum().reset_index()

#Добавляем взесы к табл с квотами:
df_mgt = pd.merge(mgt_tt_otkl, now_plng_mgt_s, on=['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень'], how='left') #2720 rows


df_mgt['p_TT']=df_mgt['p_TT'].fillna(0)
df_mgt['Взвес'] = df_mgt['Count_Quots']-df_mgt['p_TT']
df_mgt['Count_Quots']=df_mgt['Count_Quots'].fillna(0)

df_mgt['p_TT']=df_mgt['p_TT'].fillna(0)
df_mgt['Взвес'] = df_mgt['Count_Quots']-df_mgt['p_TT']

#def fo(row):
    #if row.p_TT==0:
        #return 0
    #else:
        #return row.Взвес

df_mgt['ОтклПоВзвесу'] = df_mgt.apply(fo, axis=1)
df_mgt = df_mgt.rename(columns = {'Отклонение':'Откл.', 'ОтклПоВзвесу':'ПоВзвесу'})

df_mgt=df_mgt[['ФО', 'ОБЛАСТЬ', 'Формат','Планограмма','Уровень', 'КЛАСТЕР', 'Размер', 'Откл.', 'ПоВзвесу', 'Count_Quots']]
df_mgt['ПоВзвесу']=df_mgt['ПоВзвесу'].round(1)

#df_mgt['Доля ТТ с полным наполнением или переквотой,%']=df_mgt['Доля ТТ c полным наполнением или переквотой,%'].round(0)
df_mgt['Разница взвеса от откл.']=df_mgt['ПоВзвесу']-df_mgt['Откл.']

df_mgt = df_mgt.rename(columns = {'Count_Quots':'Квоты'})

df_mgt['Квоты']=df_mgt['Квоты'].fillna(0)
df_mgt['Откл.']=df_mgt['Откл.'].fillna(0)
df_mgt['ПоВзвесу']=df_mgt['ПоВзвесу'].fillna(0)
df_mgt['Разница взвеса от откл.']=df_mgt['Разница взвеса от откл.'].fillna(0)

#Формируем таблицу для выгрузки по формату DIXY
df_mgt_dixy = df_mgt.query('Формат=="DIXY"') 
pivot_mgt_dixy=df_mgt_dixy.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mgt_dixy['Квоты']=pivot_mgt_dixy['Квоты'].fillna(0)
pivot_mgt_dixy['Откл.']=pivot_mgt_dixy['Откл.'].fillna(0)
pivot_mgt_dixy['ПоВзвесу']=pivot_mgt_dixy['ПоВзвесу'].fillna(0)
pivot_mgt_dixy['Разница взвеса от откл.']=pivot_mgt_dixy['Разница взвеса от откл.'].fillna(0)

#Формируем таблицу для выгрузки по формату GO
df_mgt_go = df_mgt.query('Формат=="GO"') 
pivot_mgt_go=df_mgt_go.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mgt_go['Квоты']=pivot_mgt_go['Квоты'].fillna(0)
pivot_mgt_go['Откл.']=pivot_mgt_go['Откл.'].fillna(0)
pivot_mgt_go['ПоВзвесу']=pivot_mgt_go['ПоВзвесу'].fillna(0)
pivot_mgt_go['Разница взвеса от откл.']=pivot_mgt_go['Разница взвеса от откл.'].fillna(0)

#Формируем таблицу для выгрузки по формату TOWER
df_mgt_t = df_mgt.query('Формат=="TOWER"') 
pivot_mgt_t=df_mgt_t.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mgt_t['Квоты']=pivot_mgt_t['Квоты'].fillna(0)
pivot_mgt_t['Откл.']=pivot_mgt_t['Откл.'].fillna(0)
pivot_mgt_t['ПоВзвесу']=pivot_mgt_t['ПоВзвесу'].fillna(0)
pivot_mgt_t['Разница взвеса от откл.']=pivot_mgt_t['Разница взвеса от откл.'].fillna(0)

pivot_mgt_dixy['ПоВзвесу']=pivot_mgt_dixy['ПоВзвесу'].round(0)
pivot_mgt_go['ПоВзвесу']=pivot_mgt_go['ПоВзвесу'].round(0)
pivot_mgt_t['ПоВзвесу']=pivot_mgt_t['ПоВзвесу'].round(0)

#Таблицы с уровнями 
#pivot_mgt_dixy.sample()
#pivot_mgt_go
#pivot_mgt_t

In [80]:
#МЯСНАЯ ГАСТРОНОМИЯ

all_plngsss_mg = all_plngsss.query('Планограмма=="МЯСНАЯ ГАСТРОНОМИЯ"') 
all_plngsss_mg.sample(3)

now_items_mg = now_items.query('Планограмма=="МЯСНАЯ ГАСТРОНОМИЯ"')  

#Посчитаем наполнение по взвесу:
now_plng_mg = pd.merge(now_items_mg, all_plngsss_mg, on=['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень'], how='left')
now_plng_mg['p_TT'] = now_plng_mg['КолТТ_присутствия'] / now_plng_mg['ТТ_max']

now_plng_mg_s = now_plng_mg.groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень']).agg('p_TT').sum().reset_index()

#Добавляем взесы к табл с квотами:
df_mg = pd.merge(mg_tt_otkl, now_plng_mg_s, on=['ФО', 'ОБЛАСТЬ', 'Формат', 'Планограмма', 'КЛАСТЕР', 'Размер', 'Уровень'], how='left') #2720 rows


df_mg['p_TT']=df_mg['p_TT'].fillna(0)
df_mg['Взвес'] = df_mg['Count_Quots']-df_mg['p_TT']
df_mg['Count_Quots']=df_mg['Count_Quots'].fillna(0)

df_mg['p_TT']=df_mg['p_TT'].fillna(0)
df_mg['Взвес'] = df_mg['Count_Quots']-df_mg['p_TT']

#def fo(row):
    #if row.p_TT==0:
        #return 0
    #else:
        #return row.Взвес

df_mg['ОтклПоВзвесу'] = df_mg.apply(fo, axis=1)
df_mg = df_mg.rename(columns = {'Отклонение':'Откл.', 'ОтклПоВзвесу':'ПоВзвесу'})

df_mg=df_mg[['ФО', 'ОБЛАСТЬ', 'Формат','Планограмма','Уровень', 'КЛАСТЕР', 'Размер', 'Откл.', 'ПоВзвесу', 'Count_Quots']]
df_mg['ПоВзвесу']=df_mg['ПоВзвесу'].round(1)

#df_mg['Доля ТТ с полным наполнением или переквотой,%']=df_mg['Доля ТТ c полным наполнением или переквотой,%'].round(0)
df_mg['Разница взвеса от откл.']=df_mg['ПоВзвесу']-df_mg['Откл.']

df_mg = df_mg.rename(columns = {'Count_Quots':'Квоты'})

df_mg['Квоты']=df_mg['Квоты'].fillna(0)
df_mg['Откл.']=df_mg['Откл.'].fillna(0)
df_mg['ПоВзвесу']=df_mg['ПоВзвесу'].fillna(0)
df_mg['Разница взвеса от откл.']=df_mg['Разница взвеса от откл.'].fillna(0)


#Формируем таблицу для выгрузки по формату DIXY
df_mg_dixy = df_mg.query('Формат=="DIXY"') 
df_mg_dixy['Размер']=df_mg_dixy['Размер'].astype(int)
pivot_mg_dixy=df_mg_dixy.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mg_dixy['Квоты']=pivot_mg_dixy['Квоты'].fillna(0)
pivot_mg_dixy['Откл.']=pivot_mg_dixy['Откл.'].fillna(0)
pivot_mg_dixy['ПоВзвесу']=pivot_mg_dixy['ПоВзвесу'].fillna(0)
pivot_mg_dixy['Разница взвеса от откл.']=pivot_mg_dixy['Разница взвеса от откл.'].fillna(0)

#Формируем таблицу для выгрузки по формату GO
df_mg_go = df_mg.query('Формат=="GO"') 
df_mg_go['Размер']=df_mg_go['Размер'].astype(int)
pivot_mg_go=df_mg_go.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mg_go['Квоты']=pivot_mg_go['Квоты'].fillna(0)
pivot_mg_go['Откл.']=pivot_mg_go['Откл.'].fillna(0)
pivot_mg_go['ПоВзвесу']=pivot_mg_go['ПоВзвесу'].fillna(0)
pivot_mg_go['Разница взвеса от откл.']=pivot_mg_go['Разница взвеса от откл.'].fillna(0)

#Формируем таблицу для выгрузки по формату TOWER
df_mg_t = df_mg.query('Формат=="TOWER"') 
df_mg_t['Размер']=df_mg_t['Размер'].astype(int)
pivot_mg_t=df_mg_t.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'Уровень', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_mg_t['Квоты']=pivot_mg_t['Квоты'].fillna(0)
pivot_mg_t['Откл.']=pivot_mg_t['Откл.'].fillna(0)
pivot_mg_t['ПоВзвесу']=pivot_mg_t['ПоВзвесу'].fillna(0)
pivot_mg_t['Разница взвеса от откл.']=pivot_mg_t['Разница взвеса от откл.'].fillna(0)

pivot_mg_dixy['ПоВзвесу']=pivot_mg_dixy['ПоВзвесу'].round(0)
pivot_mg_go['ПоВзвесу']=pivot_mg_go['ПоВзвесу'].round(0)
pivot_mg_t['ПоВзвесу']=pivot_mg_t['ПоВзвесу'].round(0)

#Таблицы с уровнями 
#pivot_mg_dixy.sample()
#pivot_mg_go
#pivot_mg_t

<div class="alert alert-success">
    <b>✅ Расчет переквот по взвесу по областям</b></div> </b></div> <a class="anchor" id="chapter811"></a>

[вначало](#chapter0)

In [81]:
#МОЛ.ГОР ЙОГУРТЫ

df_mgi_dixy_group = df_mgi_dixy.\
groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'КЛАСТЕР', 'Размер']).\
agg({'Откл.':'sum', 'ПоВзвесу':'sum', 'Квоты':'sum', 'Разница взвеса от откл.':'sum'}).\
reset_index() 

df_mgi_dixy_group['ПоВзвесу']=df_mgi_dixy_group['ПоВзвесу'].astype(int)

df_mgi_dixy_group_dixy = df_mgi_dixy_group.query('Формат=="DIXY"') 
pivot_dfc_mgi_dixy=df_mgi_dixy_group_dixy.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_dfc_mgi_dixy['Квоты']=pivot_dfc_mgi_dixy['Квоты'].fillna(0)
pivot_dfc_mgi_dixy['Откл.']=pivot_dfc_mgi_dixy['Откл.'].fillna(0)
pivot_dfc_mgi_dixy['ПоВзвесу']=pivot_dfc_mgi_dixy['ПоВзвесу'].fillna(0)

pivot_dfc_mgi_dixy['Разница взвеса от откл.']=pivot_dfc_mgi_dixy['Разница взвеса от откл.'].fillna(0)

pivot_dfc_mgi_dixy.head()

ФО                ОБЛАСТЬ КЛАСТЕР  Квоты                      Откл.  \
Размер                                           1      2      3      4     1   
0       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       H  166.0  191.0  242.0  267.0   5.0   
1       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       L  166.0    0.0  242.0  267.0  17.0   
2       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       M  166.0  191.0  242.0  267.0   6.0   
3       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       T  166.0  191.0  242.0  267.0  50.0   
4       СЗФО   НОВГОРОДСКАЯ ОБЛАСТЬ       H    0.0  191.0  242.0  267.0   0.0   

                           ПоВзвесу                      \
Размер     2      3      4        1     2      3      4   
0       11.0    6.0   21.0      5.0  11.0    6.0   21.0   
1        0.0   66.0   91.0     17.0   0.0   66.0  103.0   
2        9.0   10.0   27.0      6.0   9.0   10.0   27.0   
3       75.0  126.0  151.0     50.0  75.0  126.0  151.0   
4       27.0   49.0   44.0      0.0  27.0   49.0   44.0   

       Разница взвеса от откл.                  
Размер                       1    2    3     4  
0                          0.1  0.0  0.5   0.3  
1                          0.9  0.0  0.0  12.6  
2                          0.9  0.6  0.8   0.6  
3                          0.0  0.0  0.0   0.0  
4                          0.0  0.0  0.0   0.0

In [82]:
#МОЛ.ГОР ТРАДИ

df_mgt_dixy_group = df_mgt_dixy.\
groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'КЛАСТЕР', 'Размер']).\
agg({'Откл.':'sum', 'ПоВзвесу':'sum', 'Квоты':'sum', 'Разница взвеса от откл.':'sum'}).\
reset_index() 

df_mgt_dixy_group['ПоВзвесу']=df_mgt_dixy_group['ПоВзвесу'].astype(int)

df_mgt_dixy_group_dixy = df_mgt_dixy_group.query('Формат=="DIXY"') 
pivot_dfc_mgt_dixy=df_mgt_dixy_group_dixy.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_dfc_mgt_dixy['Квоты']=pivot_dfc_mgt_dixy['Квоты'].fillna(0)
pivot_dfc_mgt_dixy['Откл.']=pivot_dfc_mgt_dixy['Откл.'].fillna(0)
pivot_dfc_mgt_dixy['ПоВзвесу']=pivot_dfc_mgt_dixy['ПоВзвесу'].fillna(0)

pivot_dfc_mgt_dixy['Разница взвеса от откл.']=pivot_dfc_mgt_dixy['Разница взвеса от откл.'].fillna(0)

pivot_dfc_mgt_dixy.head()

ФО                ОБЛАСТЬ КЛАСТЕР Квоты                      Откл.  \
Размер                                          1      2      3      4     1   
0       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       H  95.0  115.0  135.0  162.0   0.0   
1       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       L  95.0  115.0  135.0  162.0   6.0   
2       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       M  95.0  115.0  135.0  162.0   2.0   
3       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       T  95.0  115.0  135.0  162.0  26.0   
4       СЗФО   НОВГОРОДСКАЯ ОБЛАСТЬ       H   0.0  115.0  135.0  162.0   0.0   

                         ПоВзвесу                   Разница взвеса от откл.  \
Размер     2     3     4        1     2     3     4                       1   
0        5.0   7.0  31.0      2.0   5.0   8.0  34.0                     2.4   
1       15.0  30.0  47.0     11.0  15.0  30.0  60.0                     5.4   
2        5.0  10.0  34.0      8.0  10.0  17.0  39.0                     6.0   
3       47.0  67.0  93.0     26.0  47.0  67.0  93.0                     0.0   
4       10.0  29.0  41.0      0.0  10.0  29.0  41.0                     0.0   

                        
Размер    2    3     4  
0       0.7  1.5   3.4  
1       0.0  0.0  13.7  
2       5.4  7.1   5.8  
3       0.0  0.0   0.0  
4       0.0  0.0   0.0

In [83]:
#МЯСНАЯ ГАСТРОНОМИЯ
df_mg_dixy_group = df_mg_dixy.\
groupby(['ФО', 'ОБЛАСТЬ', 'Формат', 'КЛАСТЕР', 'Размер']).\
agg({'Откл.':'sum', 'ПоВзвесу':'sum', 'Квоты':'sum', 'Разница взвеса от откл.':'sum'}).\
reset_index() 

df_mg_dixy_group['ПоВзвесу']=df_mg_dixy_group['ПоВзвесу'].astype(int)

df_mg_dixy_group_dixy = df_mg_dixy_group.query('Формат=="DIXY"') 
pivot_dfc_mg_dixy=df_mg_dixy_group_dixy.pivot_table(index=['ФО', 'ОБЛАСТЬ', 'КЛАСТЕР'],
                                values=['Откл.', 'ПоВзвесу', 'Квоты', 'Разница взвеса от откл.'],
                                columns='Размер', aggfunc='sum').reset_index()

pivot_dfc_mg_dixy['Квоты']=pivot_dfc_mg_dixy['Квоты'].fillna(0)
pivot_dfc_mg_dixy['Откл.']=pivot_dfc_mg_dixy['Откл.'].fillna(0)
pivot_dfc_mg_dixy['ПоВзвесу']=pivot_dfc_mg_dixy['ПоВзвесу'].fillna(0)

pivot_dfc_mg_dixy['Разница взвеса от откл.']=pivot_dfc_mg_dixy['Разница взвеса от откл.'].fillna(0)

pivot_dfc_mg_dixy.head()

ФО                ОБЛАСТЬ КЛАСТЕР Квоты                             \
Размер                                          1     2      3      4      5   
0       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       H   0.0   0.0  101.0  122.0  158.0   
1       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       L  73.0  96.0  101.0  122.0  158.0   
2       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       M   0.0   0.0  101.0    0.0  158.0   
3       СЗФО  ЛЕНИНГРАДСКАЯ ОБЛАСТЬ       T   0.0  96.0    0.0    0.0  158.0   
4       СЗФО   НОВГОРОДСКАЯ ОБЛАСТЬ       H   0.0   0.0    0.0    0.0  158.0   

                      ... Разница взвеса от откл.                           \
Размер      6      7  ...                       3    4    5    6    7    8   
0       179.0  210.0  ...                     0.0  0.0  0.0  0.0  1.5  0.4   
1       179.0  210.0  ...                     0.0  0.0  0.0  0.0  0.7  0.0   
2       179.0  210.0  ...                     0.0  0.0  0.0  0.0  1.2  0.6   
3         0.0    0.0  ...                     0.0  0.0  0.0  0.0  0.0  0.0   
4       179.0    0.0  ...                     0.0  0.0  0.0  0.0  0.0  0.0   

                             
Размер    9   10   11    12  
0       0.6  0.9  0.0   0.1  
1       0.0  0.0  0.0   0.0  
2       0.0  0.0  0.0  77.7  
3       0.0  0.0  0.0   0.0  
4       0.0  0.0  0.0   0.0  

[5 rows x 51 columns]

<div class="alert alert-success">
    <b>✅ Вывод на листы Excel
</div> <a class="anchor" id="chapter9"></a>

[вначало](#chapter0)

**Стили ячеек**  <a class="anchor" id="selection_9_1"></a>

In [84]:
#ЗАДАДИМ СТИЛИ ЯЧЕЕК

#заголовок таблицы
SUBTITLE_FONT = Font(name='Calibri', size=12, bold=True, italic=False, vertAlign=None, underline='none', strike=False, color='000000')
fill1 = PatternFill(start_color="feedc6", end_color="feedc6", fill_type="solid") #фон ячейки
alignment3=Alignment(horizontal='left', vertical='center', wrap_text=True) #выравнивание в ячейка (wrap_text-перенос текста)

font_wite=Font(size=9, bold=False, italic=False, color='ffffff') #стиль для первой колонки

font1=Font(name='Calibri', size=12, bold=True, italic=False, color='990000') #стиль для небольшого кол.ячеек
font2=Font(name='Arial', size=17, italic=True, color='FF0000') #стиль для небольшого кол.ячеек
font3=Font(name='Calibri', bold=True) #стиль для небольшого кол.ячеек
#font4=Font(name='Calibri', bold=True, color='FF3030')

GLOBAL_TITLE_FONT = Font(name='Calibri', size=24, bold=True, italic=False, vertAlign=None, underline='none', strike=False, color='e64117')
SZ14_FONT = Font(name='Calibri', size=11, bold=False, italic=False, vertAlign=None, underline='none', strike=False, color='000000')

alignment1=Alignment(horizontal='center', vertical='bottom', wrap_text=True) #выравнивание в ячейка (wrap_text-перенос текста) vertical='bottom'
alignment2=Alignment(horizontal='left', vertical='center', wrap_text=False) #выравнивание в ячейка (wrap_text-перенос текста)
#alignment4=Alignment(horizontal='center', vertical='center', wrap_text=False) #выравнивание в ячейка (wrap_text-перенос текста)
#alignment5=Alignment(horizontal='center', vertical='center', wrap_text=True)


#fill2 = PatternFill(start_color="FF3030", end_color="FF3030", fill_type="solid") #фон ячейки

# ГРАНИЦЫ ЯЧЕЕК
border1 = Border(
    left=Side(border_style='thin', color="FF000000"),            
    right=Side(border_style='thin', color="FF000000"),            
    top=Side(border_style='thin', color="FF000000"),
    bottom=Side(border_style='thin', color="FF000000"))

#Задаем стили для условного форматирования
cell_fill = PatternFill(start_color='abbbd1',end_color='abbbd1',fill_type='solid') # создаем объект  заливки
rule = CellIsRule(operator='equal', formula=[0], stopIfTrue=True, fill=cell_fill) # создаем правило условного форматирования "если значение ячейки 0, то залить красным"

cell_fill_Dix = PatternFill(start_color='f56b84',end_color='f56b84',fill_type='solid') 
rule_Dix = CellIsRule(operator='lessThan', formula=[0], stopIfTrue=True, fill=cell_fill_Dix) 

date_now = datetime.now().strftime('%d_%m_%Y') # получаем текущую дату

**МОЛ.ГОР ЙОГУРТЫ**  <a class="anchor" id="selection_9_2"></a>

In [85]:
#МОЛ.ГОР ЙОГУРТЫ
with pd.ExcelWriter(path_mgi_data, engine='xlsxwriter') as wb:

    #ТТ с наполнением (tt_mgi_lev #без уровней)
    tt_mgi_lev.to_excel(wb, sheet_name='ТТ с наполнением', index=False, startrow=3, freeze_panes=(4, 7) )
    sheet1 = wb.sheets['ТТ с наполнением']   
    sheet1.autofilter('A4:J'+str(tt_mgi_lev.shape[0])) 
    sheet1.set_column('A:A', 9.78) 
    sheet1.set_column('B:B', 12.78) 
    sheet1.set_column('C:C', 23.89) 
    sheet1.set_column('D:D', 8.78)
    sheet1.set_column('E:E', 24.89)  
    sheet1.set_column('F:F', 13.56) 
    sheet1.set_column('G:G', 12.44)
    sheet1.set_column('H:J', 17.56)

    #Доля ТТ  
    tt_mgi_d.to_excel(wb, sheet_name='Доля ТТ', startrow=2, freeze_panes=(5, 4) )
    sheet2 = wb.sheets['Доля ТТ']    
    sheet2.autofilter('A5:V'+str(tt_mgi_d.shape[0])) 
    sheet2.set_column('A:A', 2) 
    sheet2.set_column('B:B', 10)
    sheet2.set_column('C:C', 10) 
    sheet2.set_column('D:D', 32.33) 
    sheet2.set_column('E:V', 8.11)

    #DIXY  
    pivot_mgi_dixy.to_excel(wb, sheet_name='DIXY', startrow=3, freeze_panes=(6, 5) )
    sheet3 = wb.sheets['DIXY']    
    sheet3.autofilter('A6:U'+str(pivot_mgi_dixy.shape[0])) 
    sheet3.set_column('A:A', 2) 
    sheet3.set_column('B:B', 7)
    sheet3.set_column('C:C', 27.89) 
    sheet3.set_column('D:D', 32.33) 
    sheet3.set_column('E:E', 8.11)
    sheet3.set_column('F:V', 7.11)
    
    #DIXY итог
    pivot_dfc_mgi_dixy.to_excel(wb, sheet_name='DIXY_итог', startrow=2, freeze_panes=(5, 4) )
    sheet6 = wb.sheets['DIXY_итог']    
    sheet6.autofilter('A5:T'+str(pivot_dfc_mgi_dixy.shape[0])) 
    sheet6.set_column('A:A', 2) 
    sheet6.set_column('B:B', 7)
    sheet6.set_column('C:C', 27.89) 
    sheet6.set_column('D:D', 8.78)
    sheet6.set_column('E:P', 7.11)
        
    #GO  
    pivot_mgi_go.to_excel(wb, sheet_name='GO', startrow=2)
    sheet4 = wb.sheets['GO']    
    sheet4.autofilter('A5:I'+str(pivot_mgi_go.shape[0])) 
    sheet4.set_column('A:A', 2) 
    sheet4.set_column('B:B', 7)
    sheet4.set_column('C:C', 27.89) 
    sheet4.set_column('D:D', 32.33) 
    sheet4.set_column('E:E', 8.7)
    sheet4.set_column('F:I', 21.33)
    
    #TOWER  
    pivot_mgi_t.to_excel(wb, sheet_name='TOWER', startrow=2)
    sheet5 = wb.sheets['TOWER']    
    sheet5.autofilter('A5:I'+str(pivot_mgi_t.shape[0])) 
    sheet5.set_column('A:A', 2) 
    sheet5.set_column('B:B', 7)
    sheet5.set_column('C:C', 27.89) 
    sheet5.set_column('D:D', 32.33) 
    sheet5.set_column('E:E', 8.7)
    sheet5.set_column('F:I', 21.33) 

In [86]:
wb = load_workbook(path_mgi_data, data_only=True) # загружаем книгу, без формул

sheet1 = wb['ТТ с наполнением'] # загружаем рабочий лист
sheet2 = wb['Доля ТТ'] 
sheet3 = wb['DIXY'] # загружаем рабочий лист
sheet4 = wb['GO'] 
sheet5 = wb['TOWER'] 
sheet6 = wb['DIXY_итог'] 


#**ФОРМАТИРОВАНИЕ ВСЕХ ЛИСТОВ**

#установить цвет листа 
wb['DIXY'].sheet_properties.tabColor ='f7f9a9' #светло-желтый
wb['GO'].sheet_properties.tabColor ='f7f9a9' #светло-желтый
wb['TOWER'].sheet_properties.tabColor ='f7f9a9' #светло-желтый


#тело данных по центру
def set_alignment_in(ws, cell_range):
    align=Alignment(horizontal='center', vertical='center', wrap_text=False) 
    for row in ws[cell_range]:
        for cell in row:
            cell.alignment = align

#установка границ 
def set_border_in(ws, cell_range):
    thin = Side(border_style="thin", color="C0C0C0")
    for row in ws[cell_range]:
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)


            
#**ФОРМАТИРОВАНИЕ ОТДЕЛЬНЫХ ЛИСТОВ**

#-------------------------- ТТ с переквотой

#sheet1.sheet_view.showGridLines = False #скрыть сетку
sheet1.sheet_view.zoomScale = 80 #масштабирование в %

sheet1['A1'] = 'Список магазинов с переквотой по абсолютному отклонению от значений квот' #запись значение в ячейку А1 листа sheet2
sheet1['A1'].font=font1
sheet1['A1'].alignment = alignment2
sheet1['A2'] = tt_mgi_sum_v #запись значение в ячейку А2 листа sheet2
sheet1['A3'] = tt_mgi_sum_vn 

sheet1['A2'].font=font3
#sheet1['A3'].font=font3
sheet1.merge_cells('A1:F1') #объединение ячеек
sheet1.merge_cells('A2:F2') 
#sheet1.merge_cells('A3:F3') 

sheet1.row_dimensions[1].height = 30 # установить высоту строки


for pos in ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4', 'J4']: #заголовок таблицы
    sheet1[pos].fill = fill1
    sheet1[pos].font = SUBTITLE_FONT
    sheet1[pos].alignment = alignment3

set_alignment_in(sheet1, 'F4:J'+str(tt_mgi_lev.shape[0])+'3')  #тело данных по центру

#Определение диапазонов
cell_range_tt_A = f"{[cell for cell in sheet1['A'] if cell.value][1].coordinate}:{[cell for cell in sheet1['A'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_B = f"{[cell for cell in sheet1['B'] if cell.value][1].coordinate}:{[cell for cell in sheet1['B'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_C = f"{[cell for cell in sheet1['C'] if cell.value][1].coordinate}:{[cell for cell in sheet1['C'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_D = f"{[cell for cell in sheet1['D'] if cell.value][1].coordinate}:{[cell for cell in sheet1['D'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_E = f"{[cell for cell in sheet1['E'] if cell.value][1].coordinate}:{[cell for cell in sheet1['E'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_F = f"{[cell for cell in sheet1['F'] if cell.value][1].coordinate}:{[cell for cell in sheet1['F'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_G = f"{[cell for cell in sheet1['G'] if cell.value][1].coordinate}:{[cell for cell in sheet1['G'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_H = f"{[cell for cell in sheet1['H'] if cell.value][1].coordinate}:{[cell for cell in sheet1['H'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_I = f"{[cell for cell in sheet1['I'] if cell.value][1].coordinate}:{[cell for cell in sheet1['I'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_J = f"{[cell for cell in sheet1['J'] if cell.value][1].coordinate}:{[cell for cell in sheet1['J'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
    
#Условное форматирование
#sheet1.conditional_formatting.add(cell_range_tt_J, rule) # применяем правило для условного форматирования

#установка границ
#set_border_in(sheet1, cell_range_tt_A)  
#set_border_in(sheet1, cell_range_tt_B)
#set_border_in(sheet1, cell_range_tt_C)
#set_border_in(sheet1, cell_range_tt_D)
#set_border_in(sheet1, cell_range_tt_E)
#set_border_in(sheet1, cell_range_tt_F)
#set_border_in(sheet1, cell_range_tt_G)
#set_border_in(sheet1, cell_range_tt_H)
#set_border_in(sheet1, cell_range_tt_I)
#set_border_in(sheet1, cell_range_tt_J)



#-------------------------- Доля ТТ
#sheet2.delete_cols(1)# удалить колонку
#sheet2.column_dimensions['A'].visible = True 

for col in ['A', 'X', 'Y']: # скрыть несколько столбцов
    sheet2.column_dimensions[col].hidden = True

#sheet2.sheet_view.showGridLines = False #скрыть сетку
sheet2.sheet_view.zoomScale = 80 #масштабирование в %

sheet2['B1'] = 'Доля магазинов с переквотой по абсолютному отклонению от значений квот' 
sheet2['B1'].font=font1
sheet2['B1'].alignment = alignment2
sheet2['B2'].font=font3
#sheet2.merge_cells('A1:E1') #объединение ячеек
sheet2.merge_cells('B2:E2') #объединение ячеек

sheet2.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['E3','F3','G3','H3','I3','J3','K3','L3','M3','N3','O3','P3','Q3','R3','S3','T3','U3','V3']: #заголовок таблицы
    sheet2[pos].fill = fill1
    sheet2[pos].font = SUBTITLE_FONT
    sheet2[pos].alignment = alignment3

set_alignment_in(sheet2, 'E4:W'+str(tt_mgi_d.shape[0])+'3') #тело данных по центру
#set_border_in(sheet2, 'A5:V26') #установка границ 

#sheet2.column_dimensions.group('Q', 'V', hidden=True) # группировать столбцы с I по L с закрытой группировкой



#-------------------------- DIXY
#sheet2.delete_cols(1)# удалить колонку
#sheet2.column_dimensions['A'].visible = True 

for col in ['A', 'Y']: # скрыть несколько столбцов
    sheet3.column_dimensions[col].hidden = True

#sheet3.sheet_view.showGridLines = False #скрыть сетку
sheet3.sheet_view.zoomScale = 80 #масштабирование в %

sheet3['A1'] = 'Контроль Квот по Областям магазинов DIXY до Уровня' 
sheet3['A1'].font=font1
sheet3['A1'].alignment = alignment2

sheet3['A2'] = tt_mgi_sum_v #запись значение в ячейку А2 листа sheet2
sheet3['A3'] = tt_mgi_sum_vn 

sheet3['A2'].font=font3
sheet3.merge_cells('A1:E1') #объединение ячеек
sheet3.merge_cells('A2:E2') #объединение ячеек

sheet3.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['E4','F4','G4','H4','I4','J4','K4','L4','M4','N4','O4','P4','Q4','R4','S4','T4','U4']: #заголовок таблицы
    sheet3[pos].fill = fill1
    sheet3[pos].font = SUBTITLE_FONT
    sheet3[pos].alignment = alignment3

set_alignment_in(sheet3, 'E4:U'+str(pivot_mgi_dixy.shape[0])+'3') #тело данных по центру
#set_border_in(sheet3, 'B6:U'+str(pivot_mgi_dixy.shape[0])+'1') #установка границ 

sheet3.column_dimensions.group('F', 'I', hidden=True) # группировать столбцы с I по L с закрытой группировкой


#Определение диапазонов   

cell_range_d_B = f"{[cell for cell in sheet3['B'] if cell.value][1].coordinate}:{[cell for cell in sheet3['B'] if cell.value][-1].coordinate}" 
cell_range_d_C = f"{[cell for cell in sheet3['C'] if cell.value][1].coordinate}:{[cell for cell in sheet3['C'] if cell.value][-1].coordinate}" 
cell_range_d_D = f"{[cell for cell in sheet3['D'] if cell.value][1].coordinate}:{[cell for cell in sheet3['D'] if cell.value][-1].coordinate}" 
cell_range_d_E = f"{[cell for cell in sheet3['E'] if cell.value][1].coordinate}:{[cell for cell in sheet3['E'] if cell.value][-1].coordinate}" 
cell_range_d_F = f"{[cell for cell in sheet3['F'] if cell.value][1].coordinate}:{[cell for cell in sheet3['F'] if cell.value][-1].coordinate}" 
cell_range_d_G = f"{[cell for cell in sheet3['G'] if cell.value][1].coordinate}:{[cell for cell in sheet3['G'] if cell.value][-1].coordinate}" 
cell_range_d_H = f"{[cell for cell in sheet3['H'] if cell.value][1].coordinate}:{[cell for cell in sheet3['H'] if cell.value][-1].coordinate}" 
cell_range_d_I = f"{[cell for cell in sheet3['I'] if cell.value][1].coordinate}:{[cell for cell in sheet3['I'] if cell.value][-1].coordinate}" 
cell_range_d_J = f"{[cell for cell in sheet3['J'] if cell.value][1].coordinate}:{[cell for cell in sheet3['J'] if cell.value][-1].coordinate}" 
cell_range_d_K = f"{[cell for cell in sheet3['K'] if cell.value][1].coordinate}:{[cell for cell in sheet3['K'] if cell.value][-1].coordinate}" 
cell_range_d_L = f"{[cell for cell in sheet3['L'] if cell.value][1].coordinate}:{[cell for cell in sheet3['L'] if cell.value][-1].coordinate}" 
cell_range_d_M = f"{[cell for cell in sheet3['M'] if cell.value] [1].coordinate}:{[cell for cell in sheet3['M'] if cell.value][-1].coordinate}"
cell_range_d_N = f"{[cell for cell in sheet3['N'] if cell.value][1].coordinate}:{[cell for cell in sheet3['N'] if cell.value][-1].coordinate}"
cell_range_d_O = f"{[cell for cell in sheet3['O'] if cell.value][1].coordinate}:{[cell for cell in sheet3['O'] if cell.value][-1].coordinate}"    
cell_range_d_P = f"{[cell for cell in sheet3['P'] if cell.value][1].coordinate}:{[cell for cell in sheet3['P'] if cell.value][-1].coordinate}"
cell_range_d_Q = f"{[cell for cell in sheet3['Q'] if cell.value][1].coordinate}:{[cell for cell in sheet3['Q'] if cell.value][-1].coordinate}"
cell_range_d_R = f"{[cell for cell in sheet3['R'] if cell.value][1].coordinate}:{[cell for cell in sheet3['R'] if cell.value][-1].coordinate}"
cell_range_d_S = f"{[cell for cell in sheet3['S'] if cell.value][1].coordinate}:{[cell for cell in sheet3['S'] if cell.value][-1].coordinate}"
cell_range_d_T = f"{[cell for cell in sheet3['T'] if cell.value][1].coordinate}:{[cell for cell in sheet3['T'] if cell.value][-1].coordinate}"
cell_range_d_U = f"{[cell for cell in sheet3['U'] if cell.value][1].coordinate}:{[cell for cell in sheet3['U'] if cell.value][-1].coordinate}"

        
#Условное форматирование
sheet3.conditional_formatting.add(cell_range_d_J, rule_Dix) 
sheet3.conditional_formatting.add(cell_range_d_K, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_L, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_M, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_N, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_O, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_P, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_Q, rule_Dix)


#-------------------------- GO

for col in ['A', 'Y']: # скрыть несколько столбцов
    sheet4.column_dimensions[col].hidden = True

#sheet4.sheet_view.showGridLines = False #скрыть сетку
sheet4.sheet_view.zoomScale = 80 #масштабирование в %

sheet4['A1'] = 'Контроль Квот по Областям магазинов GO до Уровня' 
sheet4['A1'].font=font1
sheet4['A1'].alignment = alignment2
sheet4['A2'].font=font3
sheet4.merge_cells('A1:E1') #объединение ячеек
sheet4.merge_cells('A2:E2') #объединение ячеек

sheet4.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['E3','F3','G3','H3','I3']: #заголовок таблицы
    sheet4[pos].fill = fill1
    sheet4[pos].font = SUBTITLE_FONT
    sheet4[pos].alignment = alignment3

set_alignment_in(sheet4, 'E4:I'+str(pivot_mgi_go.shape[0])+'3') #тело данных по центру
#set_border_in(sheet4, 'A5:I13') #установка границ 

#Условное форматирование

column_go_G = [cell for cell in sheet4['G'] if cell.value] # получаем диапазон ячеек, исключая ячейки с значением None
#cell_range_go_G = f"{column_go_G[1].coordinate}:{column_go_G[-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_go_H = f"{[cell for cell in sheet4['H'] if cell.value][1].coordinate}:{[cell for cell in sheet4['H'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"  
    
#sheet4.conditional_formatting.add(cell_range_go_G, rule_Dix)
sheet4.conditional_formatting.add(cell_range_go_H, rule_Dix)



#-------------------------- TOWER

for col in ['A', 'Y']: # скрыть несколько столбцов
    sheet5.column_dimensions[col].hidden = True

#sheet5.sheet_view.showGridLines = False #скрыть сетку
sheet5.sheet_view.zoomScale = 80 #масштабирование в %

sheet5['A1'] = 'Контроль Квот по Областям магазинов TOWER до Уровня' 
sheet5['A1'].font=font1
sheet5['A1'].alignment = alignment2
#sheet5['A2'] = tt_mgi_sum_vn #запись значение в ячейку А2 листа sheet2
sheet5['A2'].font=font3
sheet5.merge_cells('A1:E1') #объединение ячеек
sheet5.merge_cells('A2:E2') #объединение ячеек

sheet5.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['E3','F3','G3','H3','I3']: #заголовок таблицы
    sheet5[pos].fill = fill1
    sheet5[pos].font = SUBTITLE_FONT
    sheet5[pos].alignment = alignment3

set_alignment_in(sheet5, 'E4:I'+str(pivot_mgi_t.shape[0])+'3') #тело данных по центру
#set_border_in(sheet5, 'A5:I10') #установка границ 

#Условное форматирование
 
cell_range_t_G = f"{[cell for cell in sheet5['G'] if cell.value][1].coordinate}:{[cell for cell in sheet5['G'] if cell.value][-1].coordinate}" 
cell_range_t_H = f"{[cell for cell in sheet5['H'] if cell.value] [1].coordinate}:{[cell for cell in sheet5['H'] if cell.value] [-1].coordinate}"
sheet5.conditional_formatting.add(cell_range_t_G, rule_Dix)
sheet5.conditional_formatting.add(cell_range_t_H, rule_Dix)


#-------------------------- DIXY_итог

for col in ['A', 'Y']: # скрыть несколько столбцов
    sheet6.column_dimensions[col].hidden = True

#sheet6.sheet_view.showGridLines = False #скрыть сетку
sheet6.sheet_view.zoomScale = 80 #масштабирование в %

sheet6['A1'] = 'Контроль Квот по Областям магазинов DIXY' 
sheet6['A1'].font=font1
sheet6['A1'].alignment = alignment2
sheet6['A2'].font=font3
sheet6.merge_cells('A1:E1') #объединение ячеек
sheet6.merge_cells('A2:E2') 

sheet6.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['D3','E3','F3','G3','H3','I3','J3','K3','L3','M3','N3','O3','P3','Q3','R3','S3','T3']: #заголовок таблицы
    sheet6[pos].fill = fill1
    sheet6[pos].font = SUBTITLE_FONT
    sheet6[pos].alignment = alignment3

sheet6.column_dimensions.group('E', 'H', hidden=True) # группировать столбцы с I по L с закрытой группировкой

#Условное форматирование

column_I = [cell for cell in sheet6['I'] if cell.value] # получаем диапазон ячеек, исключая ячейки с значением None
cell_range_I = f"{column_I[1].coordinate}:{column_I[-1].coordinate}" # получаем адрес диапазона вида "E2:E10"

cell_range_J = f"{[cell for cell in sheet6['J'] if cell.value][1].coordinate}:{[cell for cell in sheet6['J'] if cell.value][-1].coordinate}" 
cell_range_K = f"{[cell for cell in sheet6['K'] if cell.value][1].coordinate}:{[cell for cell in sheet6['K'] if cell.value][-1].coordinate}" 
cell_range_L = f"{[cell for cell in sheet6['L'] if cell.value][1].coordinate}:{[cell for cell in sheet6['L'] if cell.value][-1].coordinate}" 
cell_range_M = f"{[cell for cell in sheet6['M'] if cell.value][1].coordinate}:{[cell for cell in sheet6['M'] if cell.value][-1].coordinate}"
cell_range_N = f"{[cell for cell in sheet6['N'] if cell.value][1].coordinate}:{[cell for cell in sheet6['N'] if cell.value][-1].coordinate}"
cell_range_O = f"{[cell for cell in sheet6['O'] if cell.value][1].coordinate}:{[cell for cell in sheet6['O'] if cell.value][-1].coordinate}"    
cell_range_P = f"{[cell for cell in sheet6['P'] if cell.value][1].coordinate}:{[cell for cell in sheet6['P'] if cell.value][-1].coordinate}"
cell_range_E = f"{[cell for cell in sheet6['E'] if cell.value][1].coordinate}:{[cell for cell in sheet6['E'] if cell.value][-1].coordinate}"
cell_range_F = f"{[cell for cell in sheet6['F'] if cell.value][1].coordinate}:{[cell for cell in sheet6['F'] if cell.value][-1].coordinate}"
cell_range_G = f"{[cell for cell in sheet6['G'] if cell.value][1].coordinate}:{[cell for cell in sheet6['G'] if cell.value][-1].coordinate}"
cell_range_H = f"{[cell for cell in sheet6['H'] if cell.value][1].coordinate}:{[cell for cell in sheet6['H'] if cell.value][-1].coordinate}"
    
    
sheet6.conditional_formatting.add(cell_range_I, rule_Dix) # применяем правило для условного форматирования
sheet6.conditional_formatting.add(cell_range_J, rule_Dix) 
sheet6.conditional_formatting.add(cell_range_K, rule_Dix)
sheet6.conditional_formatting.add(cell_range_L, rule_Dix)
sheet6.conditional_formatting.add(cell_range_M, rule_Dix)
sheet6.conditional_formatting.add(cell_range_N, rule_Dix)
sheet6.conditional_formatting.add(cell_range_O, rule_Dix)
sheet6.conditional_formatting.add(cell_range_P, rule_Dix)


set_alignment_in(sheet6, 'D4:T'+str(pivot_dfc_mgi_dixy.shape[0])+'3') #тело данных по центру
#set_border_in(sheet6, 'B5:T'+str(pivot_dfc_mgi_dixy.shape[0])+'1') #установка границ 


#set_border_in(sheet6, cell_range_K) #установка границ 
#set_border_in(sheet6, cell_range_L)
#set_border_in(sheet6, cell_range_J)
#set_border_in(sheet6, cell_range_I)
#set_border_in(sheet6, cell_range_M)
#set_border_in(sheet6, cell_range_N)
#set_border_in(sheet6, cell_range_O)
#set_border_in(sheet6, cell_range_P)
#set_border_in(sheet6, cell_range_E)
#set_border_in(sheet6, cell_range_F)
#set_border_in(sheet6, cell_range_G)
#set_border_in(sheet6, cell_range_H)



#убрать нули
for row in sheet2.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = ''
for row in sheet3.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = '' 
for row in sheet4.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = '' 
for row in sheet5.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = '' 
for row in sheet6.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = '' 
            
wb.save(f'W:\Marketing\ОТДЕЛ УПРАВЛЕНИЯ АССОРТИМЕНТОМ\КОНТРОЛЬ КВОТ\МОЛОЧКА\МОЛ.ГОР.ЙОГУРТЫ_по Областям_{date_now}.xlsx')
wb.close()

[вначало](#chapter0)

**МОЛ.ГОР ТРАДИЦ** <a class="anchor" id="selection_9_3"></a>

In [87]:
#МОЛ.ГОР ТРАДИЦ
with pd.ExcelWriter(path_mgt_data, engine='xlsxwriter') as wb:

    #ТТ с наполнением (tt_mgt_lev #без уровней)
    tt_mgt_lev.to_excel(wb, sheet_name='ТТ с наполнением', index=False, startrow=3, freeze_panes=(4, 7) )
    sheet1 = wb.sheets['ТТ с наполнением']   
    sheet1.autofilter('A4:J'+str(tt_mgt_lev.shape[0])) 
    sheet1.set_column('A:A', 9.78) 
    sheet1.set_column('B:B', 12.78) 
    sheet1.set_column('C:C', 23.89) 
    sheet1.set_column('D:D', 8.78)
    sheet1.set_column('E:E', 24.89)  
    sheet1.set_column('F:F', 13.56) 
    sheet1.set_column('G:G', 12.44)
    sheet1.set_column('H:J', 17.56)

    #Доля ТТ  
    tt_mgt_d.to_excel(wb, sheet_name='Доля ТТ', startrow=2, freeze_panes=(5, 4) )
    sheet2 = wb.sheets['Доля ТТ']    
    sheet2.autofilter('A5:V'+str(tt_mgt_d.shape[0])) 
    sheet2.set_column('A:A', 2) 
    sheet2.set_column('B:B', 10)
    sheet2.set_column('C:C', 10) 
    sheet2.set_column('D:D', 32.33) 
    sheet2.set_column('E:V', 8.11)

    #DIXY  
    pivot_mgt_dixy.to_excel(wb, sheet_name='DIXY', startrow=3, freeze_panes=(6, 5) )
    sheet3 = wb.sheets['DIXY']    
    sheet3.autofilter('A6:U'+str(pivot_mgt_dixy.shape[0])) 
    sheet3.set_column('A:A', 2) 
    sheet3.set_column('B:B', 7)
    sheet3.set_column('C:C', 27.89) 
    sheet3.set_column('D:D', 32.33) 
    sheet3.set_column('E:E', 8.11)
    sheet3.set_column('F:V', 7.11)
    
    #DIXY итог
    pivot_dfc_mgt_dixy.to_excel(wb, sheet_name='DIXY_итог', startrow=2, freeze_panes=(5, 4) )
    sheet6 = wb.sheets['DIXY_итог']    
    sheet6.autofilter('A5:T'+str(pivot_dfc_mgt_dixy.shape[0])) 
    sheet6.set_column('A:A', 2) 
    sheet6.set_column('B:B', 7)
    sheet6.set_column('C:C', 27.89) 
    sheet6.set_column('D:D', 8.78)
    sheet6.set_column('E:P', 7.11)
        
    #GO  
    pivot_mgt_go.to_excel(wb, sheet_name='GO', startrow=2)
    sheet4 = wb.sheets['GO']    
    sheet4.autofilter('A5:I'+str(pivot_mgt_go.shape[0])) 
    sheet4.set_column('A:A', 2) 
    sheet4.set_column('B:B', 7)
    sheet4.set_column('C:C', 27.89) 
    sheet4.set_column('D:D', 32.33) 
    sheet4.set_column('E:E', 8.7)
    sheet4.set_column('F:I', 21.33)
    
    #TOWER  
    pivot_mgt_t.to_excel(wb, sheet_name='TOWER', startrow=2)
    sheet5 = wb.sheets['TOWER']    
    sheet5.autofilter('A5:I'+str(pivot_mgt_t.shape[0])) 
    sheet5.set_column('A:A', 2) 
    sheet5.set_column('B:B', 7)
    sheet5.set_column('C:C', 27.89) 
    sheet5.set_column('D:D', 32.33) 
    sheet5.set_column('E:E', 8.7)
    sheet5.set_column('F:I', 21.33) 

In [88]:
wb = load_workbook(path_mgt_data, data_only=True) # загружаем книгу, без формул

sheet1 = wb['ТТ с наполнением'] # загружаем рабочий лист
sheet2 = wb['Доля ТТ'] 
sheet3 = wb['DIXY'] # загружаем рабочий лист
sheet4 = wb['GO'] 
sheet5 = wb['TOWER'] 
sheet6 = wb['DIXY_итог'] 


#**ФОРМАТИРОВАНИЕ ВСЕХ ЛИСТОВ**

#установить цвет листа 
wb['DIXY'].sheet_properties.tabColor ='f7f9a9' #светло-желтый
wb['GO'].sheet_properties.tabColor ='f7f9a9' #светло-желтый
wb['TOWER'].sheet_properties.tabColor ='f7f9a9' #светло-желтый


#тело данных по центру
def set_alignment_in(ws, cell_range):
    align=Alignment(horizontal='center', vertical='center', wrap_text=False) 
    for row in ws[cell_range]:
        for cell in row:
            cell.alignment = align

#установка границ 
def set_border_in(ws, cell_range):
    thin = Side(border_style="thin", color="C0C0C0")
    for row in ws[cell_range]:
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)


            
#**ФОРМАТИРОВАНИЕ ОТДЕЛЬНЫХ ЛИСТОВ**

#-------------------------- ТТ с переквотой

#sheet1.sheet_view.showGridLines = False #скрыть сетку
sheet1.sheet_view.zoomScale = 80 #масштабирование в %

sheet1['A1'] = 'Список магазинов с переквотой по абсолютному отклонению от значений квот' #запись значение в ячейку А1 листа sheet2
sheet1['A1'].font=font1
sheet1['A1'].alignment = alignment2
sheet1['A2'] = tt_mgt_sum_v #запись значение в ячейку А2 листа sheet2
sheet1['A3'] = tt_mgt_sum_vn 

sheet1['A2'].font=font3
#sheet1['A3'].font=font3
sheet1.merge_cells('A1:F1') #объединение ячеек
sheet1.merge_cells('A2:F2') 
#sheet1.merge_cells('A3:F3') 

sheet1.row_dimensions[1].height = 30 # установить высоту строки


for pos in ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4', 'J4']: #заголовок таблицы
    sheet1[pos].fill = fill1
    sheet1[pos].font = SUBTITLE_FONT
    sheet1[pos].alignment = alignment3

set_alignment_in(sheet1, 'F4:J'+str(tt_mgt_lev.shape[0])+'3')  #тело данных по центру

#Определение диапазонов
cell_range_tt_A = f"{[cell for cell in sheet1['A'] if cell.value][1].coordinate}:{[cell for cell in sheet1['A'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_B = f"{[cell for cell in sheet1['B'] if cell.value][1].coordinate}:{[cell for cell in sheet1['B'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_C = f"{[cell for cell in sheet1['C'] if cell.value][1].coordinate}:{[cell for cell in sheet1['C'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_D = f"{[cell for cell in sheet1['D'] if cell.value][1].coordinate}:{[cell for cell in sheet1['D'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_E = f"{[cell for cell in sheet1['E'] if cell.value][1].coordinate}:{[cell for cell in sheet1['E'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_F = f"{[cell for cell in sheet1['F'] if cell.value][1].coordinate}:{[cell for cell in sheet1['F'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_G = f"{[cell for cell in sheet1['G'] if cell.value][1].coordinate}:{[cell for cell in sheet1['G'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_H = f"{[cell for cell in sheet1['H'] if cell.value][1].coordinate}:{[cell for cell in sheet1['H'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_I = f"{[cell for cell in sheet1['I'] if cell.value][1].coordinate}:{[cell for cell in sheet1['I'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_tt_J = f"{[cell for cell in sheet1['J'] if cell.value][1].coordinate}:{[cell for cell in sheet1['J'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
    
#Условное форматирование
#sheet1.conditional_formatting.add(cell_range_tt_J, rule) # применяем правило для условного форматирования

#установка границ
#set_border_in(sheet1, cell_range_tt_A)  
#set_border_in(sheet1, cell_range_tt_B)
#set_border_in(sheet1, cell_range_tt_C)
#set_border_in(sheet1, cell_range_tt_D)
#set_border_in(sheet1, cell_range_tt_E)
#set_border_in(sheet1, cell_range_tt_F)
#set_border_in(sheet1, cell_range_tt_G)
#set_border_in(sheet1, cell_range_tt_H)
#set_border_in(sheet1, cell_range_tt_I)
#set_border_in(sheet1, cell_range_tt_J)



#-------------------------- Доля ТТ
#sheet2.delete_cols(1)# удалить колонку
#sheet2.column_dimensions['A'].visible = True 

for col in ['A', 'X', 'Y']: # скрыть несколько столбцов
    sheet2.column_dimensions[col].hidden = True

#sheet2.sheet_view.showGridLines = False #скрыть сетку
sheet2.sheet_view.zoomScale = 80 #масштабирование в %

sheet2['B1'] = 'Доля магазинов с переквотой по абсолютному отклонению от значений квот' 
sheet2['B1'].font=font1
sheet2['B1'].alignment = alignment2
sheet2['B2'].font=font3
#sheet2.merge_cells('A1:E1') #объединение ячеек
sheet2.merge_cells('B2:E2') #объединение ячеек

sheet2.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['E3','F3','G3','H3','I3','J3','K3','L3','M3','N3','O3','P3','Q3','R3','S3','T3','U3','V3']: #заголовок таблицы
    sheet2[pos].fill = fill1
    sheet2[pos].font = SUBTITLE_FONT
    sheet2[pos].alignment = alignment3

set_alignment_in(sheet2, 'E4:W'+str(tt_mgt_d.shape[0])+'3') #тело данных по центру
#set_border_in(sheet2, 'A5:V26') #установка границ 

#sheet2.column_dimensions.group('Q', 'V', hidden=True) # группировать столбцы с I по L с закрытой группировкой



#-------------------------- DIXY
#sheet2.delete_cols(1)# удалить колонку
#sheet2.column_dimensions['A'].visible = True 

for col in ['A', 'Y']: # скрыть несколько столбцов
    sheet3.column_dimensions[col].hidden = True

#sheet3.sheet_view.showGridLines = False #скрыть сетку
sheet3.sheet_view.zoomScale = 80 #масштабирование в %

sheet3['A1'] = 'Контроль Квот по Областям магазинов DIXY до Уровня' 
sheet3['A1'].font=font1
sheet3['A1'].alignment = alignment2

sheet3['A2'] = tt_mgt_sum_v #запись значение в ячейку А2 листа sheet2
sheet3['A3'] = tt_mgt_sum_vn 

sheet3['A2'].font=font3
sheet3.merge_cells('A1:E1') #объединение ячеек
sheet3.merge_cells('A2:E2') #объединение ячеек

sheet3.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['E4','F4','G4','H4','I4','J4','K4','L4','M4','N4','O4','P4','Q4','R4','S4','T4','U4']: #заголовок таблицы
    sheet3[pos].fill = fill1
    sheet3[pos].font = SUBTITLE_FONT
    sheet3[pos].alignment = alignment3

set_alignment_in(sheet3, 'E4:U'+str(pivot_mgt_dixy.shape[0])+'3') #тело данных по центру
#set_border_in(sheet3, 'B6:U'+str(pivot_mgt_dixy.shape[0])+'1') #установка границ 

sheet3.column_dimensions.group('F', 'I', hidden=True) # группировать столбцы с I по L с закрытой группировкой


#Определение диапазонов   

cell_range_d_B = f"{[cell for cell in sheet3['B'] if cell.value][1].coordinate}:{[cell for cell in sheet3['B'] if cell.value][-1].coordinate}" 
cell_range_d_C = f"{[cell for cell in sheet3['C'] if cell.value][1].coordinate}:{[cell for cell in sheet3['C'] if cell.value][-1].coordinate}" 
cell_range_d_D = f"{[cell for cell in sheet3['D'] if cell.value][1].coordinate}:{[cell for cell in sheet3['D'] if cell.value][-1].coordinate}" 
cell_range_d_E = f"{[cell for cell in sheet3['E'] if cell.value][1].coordinate}:{[cell for cell in sheet3['E'] if cell.value][-1].coordinate}" 
cell_range_d_F = f"{[cell for cell in sheet3['F'] if cell.value][1].coordinate}:{[cell for cell in sheet3['F'] if cell.value][-1].coordinate}" 
cell_range_d_G = f"{[cell for cell in sheet3['G'] if cell.value][1].coordinate}:{[cell for cell in sheet3['G'] if cell.value][-1].coordinate}" 
cell_range_d_H = f"{[cell for cell in sheet3['H'] if cell.value][1].coordinate}:{[cell for cell in sheet3['H'] if cell.value][-1].coordinate}" 
cell_range_d_I = f"{[cell for cell in sheet3['I'] if cell.value][1].coordinate}:{[cell for cell in sheet3['I'] if cell.value][-1].coordinate}" 
cell_range_d_J = f"{[cell for cell in sheet3['J'] if cell.value][1].coordinate}:{[cell for cell in sheet3['J'] if cell.value][-1].coordinate}" 
cell_range_d_K = f"{[cell for cell in sheet3['K'] if cell.value][1].coordinate}:{[cell for cell in sheet3['K'] if cell.value][-1].coordinate}" 
cell_range_d_L = f"{[cell for cell in sheet3['L'] if cell.value][1].coordinate}:{[cell for cell in sheet3['L'] if cell.value][-1].coordinate}" 
cell_range_d_M = f"{[cell for cell in sheet3['M'] if cell.value] [1].coordinate}:{[cell for cell in sheet3['M'] if cell.value][-1].coordinate}"
cell_range_d_N = f"{[cell for cell in sheet3['N'] if cell.value][1].coordinate}:{[cell for cell in sheet3['N'] if cell.value][-1].coordinate}"
cell_range_d_O = f"{[cell for cell in sheet3['O'] if cell.value][1].coordinate}:{[cell for cell in sheet3['O'] if cell.value][-1].coordinate}"    
cell_range_d_P = f"{[cell for cell in sheet3['P'] if cell.value][1].coordinate}:{[cell for cell in sheet3['P'] if cell.value][-1].coordinate}"
cell_range_d_Q = f"{[cell for cell in sheet3['Q'] if cell.value][1].coordinate}:{[cell for cell in sheet3['Q'] if cell.value][-1].coordinate}"
cell_range_d_R = f"{[cell for cell in sheet3['R'] if cell.value][1].coordinate}:{[cell for cell in sheet3['R'] if cell.value][-1].coordinate}"
cell_range_d_S = f"{[cell for cell in sheet3['S'] if cell.value][1].coordinate}:{[cell for cell in sheet3['S'] if cell.value][-1].coordinate}"
cell_range_d_T = f"{[cell for cell in sheet3['T'] if cell.value][1].coordinate}:{[cell for cell in sheet3['T'] if cell.value][-1].coordinate}"
cell_range_d_U = f"{[cell for cell in sheet3['U'] if cell.value][1].coordinate}:{[cell for cell in sheet3['U'] if cell.value][-1].coordinate}"

        
#Условное форматирование
sheet3.conditional_formatting.add(cell_range_d_J, rule_Dix) 
sheet3.conditional_formatting.add(cell_range_d_K, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_L, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_M, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_N, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_O, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_P, rule_Dix)
sheet3.conditional_formatting.add(cell_range_d_Q, rule_Dix)


#-------------------------- GO

for col in ['A', 'Y']: # скрыть несколько столбцов
    sheet4.column_dimensions[col].hidden = True

#sheet4.sheet_view.showGridLines = False #скрыть сетку
sheet4.sheet_view.zoomScale = 80 #масштабирование в %

sheet4['A1'] = 'Контроль Квот по Областям магазинов GO до Уровня' 
sheet4['A1'].font=font1
sheet4['A1'].alignment = alignment2
sheet4['A2'].font=font3
sheet4.merge_cells('A1:E1') #объединение ячеек
sheet4.merge_cells('A2:E2') #объединение ячеек

sheet4.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['E3','F3','G3','H3','I3']: #заголовок таблицы
    sheet4[pos].fill = fill1
    sheet4[pos].font = SUBTITLE_FONT
    sheet4[pos].alignment = alignment3

set_alignment_in(sheet4, 'E4:I'+str(pivot_mgt_go.shape[0])+'3') #тело данных по центру
#set_border_in(sheet4, 'A5:I13') #установка границ 

#Условное форматирование

column_go_G = [cell for cell in sheet4['G'] if cell.value] # получаем диапазон ячеек, исключая ячейки с значением None
#cell_range_go_G = f"{column_go_G[1].coordinate}:{column_go_G[-1].coordinate}" # получаем адрес диапазона вида "E2:E10"
cell_range_go_H = f"{[cell for cell in sheet4['H'] if cell.value][1].coordinate}:{[cell for cell in sheet4['H'] if cell.value][-1].coordinate}" # получаем адрес диапазона вида "E2:E10"  
    
#sheet4.conditional_formatting.add(cell_range_go_G, rule_Dix)
sheet4.conditional_formatting.add(cell_range_go_H, rule_Dix)



#-------------------------- TOWER

for col in ['A', 'Y']: # скрыть несколько столбцов
    sheet5.column_dimensions[col].hidden = True

#sheet5.sheet_view.showGridLines = False #скрыть сетку
sheet5.sheet_view.zoomScale = 80 #масштабирование в %

sheet5['A1'] = 'Контроль Квот по Областям магазинов TOWER до Уровня' 
sheet5['A1'].font=font1
sheet5['A1'].alignment = alignment2
#sheet5['A2'] = tt_mgt_sum_vn #запись значение в ячейку А2 листа sheet2
sheet5['A2'].font=font3
sheet5.merge_cells('A1:E1') #объединение ячеек
sheet5.merge_cells('A2:E2') #объединение ячеек

sheet5.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['E3','F3','G3','H3','I3']: #заголовок таблицы
    sheet5[pos].fill = fill1
    sheet5[pos].font = SUBTITLE_FONT
    sheet5[pos].alignment = alignment3

set_alignment_in(sheet5, 'E4:I'+str(pivot_mgt_t.shape[0])+'3') #тело данных по центру
#set_border_in(sheet5, 'A5:I10') #установка границ 

#Условное форматирование
 
cell_range_t_G = f"{[cell for cell in sheet5['G'] if cell.value][1].coordinate}:{[cell for cell in sheet5['G'] if cell.value][-1].coordinate}" 
cell_range_t_H = f"{[cell for cell in sheet5['H'] if cell.value] [1].coordinate}:{[cell for cell in sheet5['H'] if cell.value] [-1].coordinate}"
sheet5.conditional_formatting.add(cell_range_t_G, rule_Dix)
sheet5.conditional_formatting.add(cell_range_t_H, rule_Dix)


#-------------------------- DIXY_итог

for col in ['A', 'Y']: # скрыть несколько столбцов
    sheet6.column_dimensions[col].hidden = True

#sheet6.sheet_view.showGridLines = False #скрыть сетку
sheet6.sheet_view.zoomScale = 80 #масштабирование в %

sheet6['A1'] = 'Контроль Квот по Областям магазинов DIXY' 
sheet6['A1'].font=font1
sheet6['A1'].alignment = alignment2
sheet6['A2'].font=font3
sheet6.merge_cells('A1:E1') #объединение ячеек
sheet6.merge_cells('A2:E2') 

sheet6.row_dimensions[1].height = 30 # установить высоту строки 

for pos in ['D3','E3','F3','G3','H3','I3','J3','K3','L3','M3','N3','O3','P3','Q3','R3','S3','T3']: #заголовок таблицы
    sheet6[pos].fill = fill1
    sheet6[pos].font = SUBTITLE_FONT
    sheet6[pos].alignment = alignment3

sheet6.column_dimensions.group('E', 'H', hidden=True) # группировать столбцы с I по L с закрытой группировкой

#Условное форматирование

column_I = [cell for cell in sheet6['I'] if cell.value] # получаем диапазон ячеек, исключая ячейки с значением None
cell_range_I = f"{column_I[1].coordinate}:{column_I[-1].coordinate}" # получаем адрес диапазона вида "E2:E10"

cell_range_J = f"{[cell for cell in sheet6['J'] if cell.value][1].coordinate}:{[cell for cell in sheet6['J'] if cell.value][-1].coordinate}" 
cell_range_K = f"{[cell for cell in sheet6['K'] if cell.value][1].coordinate}:{[cell for cell in sheet6['K'] if cell.value][-1].coordinate}" 
cell_range_L = f"{[cell for cell in sheet6['L'] if cell.value][1].coordinate}:{[cell for cell in sheet6['L'] if cell.value][-1].coordinate}" 
cell_range_M = f"{[cell for cell in sheet6['M'] if cell.value][1].coordinate}:{[cell for cell in sheet6['M'] if cell.value][-1].coordinate}"
cell_range_N = f"{[cell for cell in sheet6['N'] if cell.value][1].coordinate}:{[cell for cell in sheet6['N'] if cell.value][-1].coordinate}"
cell_range_O = f"{[cell for cell in sheet6['O'] if cell.value][1].coordinate}:{[cell for cell in sheet6['O'] if cell.value][-1].coordinate}"    
cell_range_P = f"{[cell for cell in sheet6['P'] if cell.value][1].coordinate}:{[cell for cell in sheet6['P'] if cell.value][-1].coordinate}"
cell_range_E = f"{[cell for cell in sheet6['E'] if cell.value][1].coordinate}:{[cell for cell in sheet6['E'] if cell.value][-1].coordinate}"
cell_range_F = f"{[cell for cell in sheet6['F'] if cell.value][1].coordinate}:{[cell for cell in sheet6['F'] if cell.value][-1].coordinate}"
cell_range_G = f"{[cell for cell in sheet6['G'] if cell.value][1].coordinate}:{[cell for cell in sheet6['G'] if cell.value][-1].coordinate}"
cell_range_H = f"{[cell for cell in sheet6['H'] if cell.value][1].coordinate}:{[cell for cell in sheet6['H'] if cell.value][-1].coordinate}"
    
    
##sheet6.conditional_formatting.add(cell_range_I, rule_Dix) # применяем правило для условного форматирования
sheet6.conditional_formatting.add(cell_range_J, rule_Dix) 
sheet6.conditional_formatting.add(cell_range_K, rule_Dix)
sheet6.conditional_formatting.add(cell_range_L, rule_Dix)
sheet6.conditional_formatting.add(cell_range_M, rule_Dix)
##sheet6.conditional_formatting.add(cell_range_N, rule_Dix)
sheet6.conditional_formatting.add(cell_range_O, rule_Dix)
sheet6.conditional_formatting.add(cell_range_P, rule_Dix)


set_alignment_in(sheet6, 'D4:T'+str(pivot_dfc_mgt_dixy.shape[0])+'3') #тело данных по центру
#set_border_in(sheet6, 'B5:T'+str(pivot_dfc_mgt_dixy.shape[0])+'1') #установка границ 


#set_border_in(sheet6, cell_range_K) #установка границ 
#set_border_in(sheet6, cell_range_L)
#set_border_in(sheet6, cell_range_J)
#set_border_in(sheet6, cell_range_I)
#set_border_in(sheet6, cell_range_M)
#set_border_in(sheet6, cell_range_N)
#set_border_in(sheet6, cell_range_O)
#set_border_in(sheet6, cell_range_P)
#set_border_in(sheet6, cell_range_E)
#set_border_in(sheet6, cell_range_F)
#set_border_in(sheet6, cell_range_G)
#set_border_in(sheet6, cell_range_H)



#убрать нули
for row in sheet2.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = ''
for row in sheet3.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = '' 
for row in sheet4.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = '' 
for row in sheet5.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = '' 
for row in sheet6.iter_rows():
    for cell in row:
        if cell.value  == 0:
            cell.value = '' 
            
wb.save(f'W:\Marketing\ОТДЕЛ УПРАВЛЕНИЯ АССОРТИМЕНТОМ\КОНТРОЛЬ КВОТ\МОЛОЧКА\МОЛ.ГОР.ТРАДИ_по Областям_{date_now}.xlsx')
wb.close()

[вначало](#chapter0)

Отправка почты

The End <a class="anchor" id="chapter10"></a>

[вначало](#chapter0)